In [17]:
import re
import random
import difflib
from collections import defaultdict

class HospitalChatbot:
    def __init__(self):
        # Initialize response categories and regex patterns
        self.categories = {
            "appointment": self.get_appointment_patterns(),
            "billing": self.get_billing_patterns(),
            "pharmacy": self.get_pharmacy_patterns(),
            "lab_tests": self.get_lab_test_patterns(),
            "emergency": self.get_emergency_patterns(),
            "admission": self.get_admission_patterns(),
            "discharge": self.get_discharge_patterns(),
            "insurance": self.get_insurance_patterns(),
            "visitor": self.get_visitor_patterns(),
            "cafeteria": self.get_cafeteria_patterns(),
            "volunteer": self.get_volunteer_patterns(),
            "feedback": self.get_feedback_patterns(),
            "complaints": self.get_complaint_patterns(),
            "doctor_info": self.get_doctor_info_patterns(),
            "facilities": self.get_facilities_patterns(),
            "covid": self.get_covid_patterns(),
            "contact": self.get_contact_patterns(),
            "working_hours": self.get_working_hours_patterns(),
            "parking": self.get_parking_patterns(),
            "directions": self.get_directions_patterns(),
            "greeting": self.get_greeting_patterns(),
            "goodbye": self.get_goodbye_patterns(),
            "thanks": self.get_thanks_patterns(),
            "general": self.get_general_patterns()
        }

        # Compile all regex patterns
        self.compiled_patterns = self.compile_patterns()

        # Common misspellings dictionary for basic spelling correction
        self.spelling_corrections = self.get_spelling_corrections()

        # Sample patient data for personalization (in a real system, this would come from a database)
        self.patient_data = {
            "John Doe": {"appointments": ["03/15/2025, 10:00 AM, Dr. Smith"], "medications": ["Lisinopril 10mg"]},
            "Jane Smith": {"appointments": ["03/10/2025, 2:30 PM, Dr. Johnson"], "medications": ["Metformin 500mg"]}
        }

    def compile_patterns(self):
        compiled = []
        for category, patterns in self.categories.items():
            for pattern_dict in patterns:
                regex = pattern_dict["regex"]
                response_fn = pattern_dict["response"]
                compiled.append((re.compile(regex, re.IGNORECASE), response_fn, category))
        return compiled

    def correct_spelling(self, text):
        words = text.split()
        corrected_words = []

        for word in words:
            if word.lower() in self.spelling_corrections:
                corrected_words.append(self.spelling_corrections[word.lower()])
            else:
                # Check for close matches
                close_matches = difflib.get_close_matches(word.lower(), self.spelling_corrections.keys(), n=1, cutoff=0.8)
                if close_matches:
                    corrected_words.append(self.spelling_corrections[close_matches[0]])
                else:
                    corrected_words.append(word)

        return " ".join(corrected_words)

    def process_query(self, query, user_name=None):
        # Step 1: Apply spelling correction
        corrected_query = self.correct_spelling(query)

        # Step 2: Try to match against regex patterns
        for pattern, response_fn, category in self.compiled_patterns:
            match = pattern.search(corrected_query)
            if match:
                # If we have a match, call the response function with the match object and user name
                return response_fn(match, user_name)

        # Step 3: If no match found, provide a catch-all response
        return self.catch_all_response(corrected_query)

    def catch_all_response(self, query):
        responses = [
            "I'm not sure I understand your question about the hospital. Could you please rephrase it?",
            "I don't have information on that specific topic. Could you try asking in a different way?",
            "I'm sorry, but I couldn't understand your query. Please try to be more specific about what hospital service you need help with.",
            "I didn't quite catch that. Are you asking about appointments, billing, medical services, or something else?",
            "I want to help, but I'm not understanding your request. Could you provide more details about what you need assistance with?"
        ]
        return random.choice(responses)

    #========== PATTERN DEFINITIONS ==========

    def get_appointment_patterns(self):
        return [
            {
                "regex": r"(?:how|what)(?:'s| is| are) (?:the|your) (?:process|steps|procedure) (?:to|for) (?:book|schedule|make) (?:an |a )?appointment",
                "response": lambda match, user: "To schedule an appointment at our hospital, you can:\n1. Call our appointment line at (555) 123-4567\n2. Use our patient portal at hospital.example.com\n3. Visit the front desk in person\n\nYou'll need your insurance information and patient ID if you're a returning patient."
            },
            {
                "regex": r"(?:can|could) (?:I|you) (?:book|schedule|make) (?:an |a )?appointment (?:with|for) (Dr\.|Doctor) ([A-Za-z]+)",
                "response": lambda match, user: f"Yes, you can schedule an appointment with {match.group(1)} {match.group(2)}. Please call our appointment desk at (555) 123-4567 or use our online patient portal. When would you prefer to see {match.group(1)} {match.group(2)}?"
            },
            {
                "regex": r"(?:show|tell|list|what are) my (?:upcoming |scheduled |future )?appointments",
                "response": lambda match, user: self.get_user_appointments(user)
            },
            {
                "regex": r"(?:I want to|I need to|I would like to|I'd like to) (?:cancel|reschedule) (?:my|an) appointment",
                "response": lambda match, user: "To cancel or reschedule an appointment, please call (555) 123-4567 at least 24 hours in advance. Alternatively, you can log into your patient portal and modify your appointment there. Is there a specific appointment you need to change?"
            },
            {
                "regex": r"(?:what|which) (?:doctors|specialists|physicians) (?:are|do you have) (?:available|specialized) (?:for|in) ([\w\s]+)",
                "response": lambda match, user: self.get_specialists_response(match.group(1))
            },
            {
                "regex": r"(?:how far|how long) in advance (?:should|do I need to|can I|must I) (?:book|schedule|make) (?:an |a )?appointment",
                "response": lambda match, user: "For routine check-ups and non-urgent care, we recommend scheduling 2-3 weeks in advance. Specialist appointments may require 4-6 weeks advance booking. Urgent care visits can often be accommodated same-day or next-day depending on availability."
            },
            {
                "regex": r"(?:what|which) (?:documents|things|paperwork|items) (?:do I need|should I bring|are required) (?:for|to) (?:my|an) appointment",
                "response": lambda match, user: "Please bring the following to your appointment:\n1. Photo ID\n2. Insurance card\n3. List of current medications\n4. Medical history forms (if new patient)\n5. Any relevant medical records or test results\n6. Payment method for copays\n\nArriving 15 minutes early helps with the check-in process."
            },
            {
                "regex": r"(?:how long|what duration|how much time) (?:does|will|is) (?:an|a typical|the average) appointment (?:take|last|require)",
                "response": lambda match, user: "Most routine appointments last 15-30 minutes. Initial consultations and specialist appointments typically take 30-60 minutes. Complex procedures may require more time. We'll do our best to respect your schedule, though occasional delays can occur if emergencies arise."
            },
            {
                "regex": r"(?:what happens|what should I expect|what's the process) (?:if|when) (?:I|someone) miss(?:es)? (?:an|my|their) appointment",
                "response": lambda match, user: "If you miss an appointment without prior notification, our policy includes:\n1. A missed appointment fee of $25-$50 may apply (varies by department)\n2. Documentation in your medical record\n3. After three missed appointments, we may require prepayment for future visits\n\nWe understand emergencies happen, so please call us as soon as possible if you can't make your appointment."
            },
            {
                "regex": r"(?:can|do) (?:I|you|we) (?:book|schedule|make) (?:an |a )?appointment (?:online|through the website|through the portal|through the app)",
                "response": lambda match, user: "Yes, you can schedule appointments online through our patient portal at hospital.example.com/portal. After logging in, select 'Schedule Appointment,' choose your provider and preferred time slots. The system will show available options and allow you to confirm your selection. You'll receive an email confirmation afterward."
            },
            # Add 90+ more appointment patterns here
        ]

    def get_billing_patterns(self):
        return [
            {
                "regex": r"(?:how|what)(?:'s| is) (?:the|your) (?:process|procedure) (?:for|to) (?:pay|settle|clear) (?:my|a|the) (?:bill|invoice|payment)",
                "response": lambda match, user: "You can pay your hospital bill through several methods:\n1. Online through our patient portal\n2. By phone at (555) 987-6543\n3. By mail to Hospital Billing Dept, 123 Main St\n4. In person at our billing office\n\nWe accept credit cards, checks, and payment plans for balances over $200."
            },
            {
                "regex": r"(?:I have|there is|there's) (?:a|an) (?:issue|problem|error|mistake|discrepancy) (?:with|in|on) my bill",
                "response": lambda match, user: "I'm sorry to hear about the issue with your bill. Please contact our billing department at (555) 987-6543 Monday-Friday, 8am-5pm. Have your account number and the specific concerns ready. Our billing specialists can review charges, explain any confusing items, and correct errors if needed."
            },
            {
                "regex": r"(?:do you|does the hospital) (?:offer|provide|have) (?:payment plans|installment plans|payment options)",
                "response": lambda match, user: "Yes, we offer flexible payment plans for medical bills. For balances over $200, you can spread payments over 3-24 months, depending on the total amount. There's no interest for plans completed within 12 months. To set up a payment plan, contact our financial counselors at (555) 987-6543 or visit the billing office."
            },
            {
                "regex": r"(?:what|which) (?:insurance|insurance plans|insurance providers) (?:do you|does the hospital) (?:accept|take|work with)",
                "response": lambda match, user: "We accept most major insurance plans including:\n- Blue Cross Blue Shield\n- Aetna\n- Cigna\n- UnitedHealthcare\n- Medicare and Medicaid\n- Humana\n- Kaiser Permanente\n\nFor specific questions about your insurance coverage, please contact our insurance verification department at (555) 456-7890."
            },
            {
                "regex": r"(?:how|where|what) (?:can|do) (?:I|you) (?:view|see|access|get|obtain) (?:my|the) (?:bill|invoice|statement) (?:online|electronically|digitally)",
                "response": lambda match, user: "You can view and pay your bill online through our secure patient portal at hospital.example.com/portal. After logging in with your credentials, select 'Billing' from the menu. There, you'll see all current and past statements, payment history, and options to make payments. If you need help accessing your online account, call (555) 987-6543."
            },
            {
                "regex": r"(?:how much|what) (?:does|is the cost of|will it cost for) (?:a|an) ([a-zA-Z\s]+)(?:\?)?",
                "response": lambda match, user: self.get_procedure_cost_estimate(match.group(1))
            },
            {
                "regex": r"(?:what|which) (?:financial assistance|financial aid|charity care|help paying) (?:options|programs|resources) (?:are available|do you offer|does the hospital provide)",
                "response": lambda match, user: "We offer several financial assistance programs:\n1. Charity Care for patients below 300% of federal poverty level\n2. Sliding scale discounts based on income\n3. Catastrophic care assistance for large bills\n4. Payment plans without interest\n5. Help applying for Medicaid\n\nTo apply, request a Financial Assistance Application from our Financial Counseling office at (555) 123-7890."
            },
            {
                "regex": r"(?:when|how often|how frequently) (?:will I|do you|does the hospital) (?:receive|send|issue) (?:bills|invoices|statements)",
                "response": lambda match, user: "Hospital bills are typically issued within 30 days after services are rendered or after insurance processing. Monthly statements are sent if you have an outstanding balance. You can choose paper statements by mail or electronic statements through our patient portal. If you have a payment plan, statements will show your monthly payment amount and remaining balance."
            },
            {
                "regex": r"(?:what|which) (?:information|details|data) (?:is|are) (?:included|shown|present) (?:on|in) (?:my|the) (?:bill|invoice|statement)",
                "response": lambda match, user: "Your hospital bill includes:\n1. Patient and guarantor information\n2. Service dates and descriptions\n3. Procedure and diagnosis codes\n4. Itemized charges for services\n5. Insurance payments and adjustments\n6. Patient responsibility amount\n7. Payment due date\n8. Payment options\n\nIf you need a more detailed explanation of any charges, our billing specialists can help at (555) 987-6543."
            },
            {
                "regex": r"(?:how|what) (?:is|about) (?:the|your) (?:self-pay|uninsured|no insurance|cash) (?:discount|rate|pricing)",
                "response": lambda match, user: "We offer a self-pay discount of 30-40% for patients without insurance who pay at the time of service or establish a payment plan. Additional financial assistance may be available based on income. Our financial counselors can provide a cost estimate before your procedure and help determine your eligibility for discounts. Contact them at (555) 234-5678."
            },
            # Add 90+ more billing patterns here
        ]

    def get_pharmacy_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:pharmacy|hospital pharmacy) (?:hours|timing|schedule)",
                "response": lambda match, user: "Our hospital pharmacy is open:\n- Monday-Friday: 8:00 AM to 9:00 PM\n- Saturday: 9:00 AM to 6:00 PM\n- Sunday: 10:00 AM to 5:00 PM\n\nThe inpatient pharmacy for admitted patients operates 24/7. The outpatient pharmacy in the main lobby serves all visitors and discharged patients."
            },
            {
                "regex": r"(?:can|do) (?:I|you) (?:get|refill|fill) (?:my|a) prescription (?:at|from|through) (?:the|your) (?:hospital|hospital's) pharmacy",
                "response": lambda match, user: "Yes, you can fill or refill prescriptions at our hospital pharmacy. We accept prescriptions from any provider, not just our hospital doctors. For refills, you can call (555) 234-6789, use our mobile app, or visit in person. Please allow 20-30 minutes for your prescription to be filled unless it's a specialty medication."
            },
            {
                "regex": r"(?:how|what)(?:'s| is) (?:the|your) (?:process|procedure) (?:for|to) (?:refill|renew) (?:my|a) prescription",
                "response": lambda match, user: "To refill a prescription at our pharmacy:\n1. Call our automated refill line at (555) 234-6789 and enter your prescription number\n2. Use our hospital app and select 'Prescription Refill'\n3. Visit the pharmacy in person with your prescription bottle\n4. Ask your doctor to send a new refill authorization electronically\n\nPlease request refills 2-3 days before you run out of medication."
            },
            {
                "regex": r"(?:do you|does the pharmacy) (?:deliver|ship|mail|send) (?:medications|prescriptions|medicine)",
                "response": lambda match, user: "Yes, our pharmacy offers medication delivery services through several options:\n1. Free home delivery within a 10-mile radius for orders over $25\n2. Mail order service for maintenance medications (usually 90-day supplies)\n3. Bedside delivery for patients being discharged\n\nFor delivery, call (555) 234-6789. Please allow 1-2 business days for local delivery and 3-5 days for mail orders."
            },
            {
                "regex": r"(?:what|which) (?:insurance|insurance plans|insurance providers) (?:do you|does the pharmacy) (?:accept|take|work with)",
                "response": lambda match, user: "Our pharmacy accepts most major prescription insurance plans including:\n- Express Scripts\n- CVS Caremark\n- OptumRx\n- Blue Cross Blue Shield\n- Medicare Part D\n- Medicaid\n- Tricare\n\nWe also work with most major medical insurance providers. If you have questions about your specific coverage, please call our pharmacy team at (555) 234-6789."
            },
            {
                "regex": r"(?:do you|does the pharmacy) (?:have|carry|stock) ([a-zA-Z\s]+)(?:\?)?",
                "response": lambda match, user: self.check_medication_availability(match.group(1))
            },
            {
                "regex": r"(?:how long|how much time) (?:does it take|will it take) (?:to|for) (?:fill|prepare|get) (?:a|my) prescription",
                "response": lambda match, user: "For most standard medications, our pharmacy can fill your prescription in 20-30 minutes. During peak hours (11am-1pm and 4pm-6pm), wait times may extend to 45-60 minutes. Specialty medications or compounded prescriptions may take 24-48 hours. We offer a text notification service to alert you when your prescription is ready."
            },
            {
                "regex": r"(?:can|do) (?:I|you) (?:transfer|move) (?:my|a) prescription (?:from|to) (?:another|different|outside) pharmacy",
                "response": lambda match, user: "Yes, we can transfer prescriptions to or from other pharmacies. To transfer a prescription to our pharmacy, simply provide us with:\n1. Your name, DOB, and contact information\n2. The name and phone number of your current pharmacy\n3. The name of the medication and prescription number if available\n\nCall our pharmacy team at (555) 234-6789 to initiate the transfer."
            },
            {
                "regex": r"(?:do you|does the pharmacy) (?:offer|provide|have) (?:generic|cheaper|less expensive|affordable) (?:alternatives|options|substitutes)",
                "response": lambda match, user: "Yes, our pharmacy provides generic alternatives when available. Our pharmacists automatically check for FDA-approved generic substitutions that can save you money unless your doctor has specified 'dispense as written.' We also offer a prescription savings program for uninsured patients that provides many generics at reduced prices, starting at $4 for a 30-day supply."
            },
            {
                "regex": r"(?:what|which) (?:medications|medicines|drugs) (?:are|does|do you) (?:carry|stock|have) (?:for|to treat) ([a-zA-Z\s]+)",
                "response": lambda match, user: self.get_medications_for_condition(match.group(1))
            },
            # Add 90+ more pharmacy patterns here
        ]

    def get_lab_test_patterns(self):
        return [
            {
                "regex": r"(?:how|what)(?:'s| is) (?:the|your) (?:process|procedure) (?:for|to) (?:get|schedule|book) (?:a|an) (?:lab test|labwork|laboratory test|blood test)",
                "response": lambda match, user: "To schedule a lab test at our hospital:\n1. Get a test order from your doctor (electronic or paper)\n2. Call our lab scheduling line at (555) 345-6789\n3. Schedule an appointment time (or walk in for routine tests)\n4. Arrive 15 minutes early with your ID and insurance card\n5. Check in at the laboratory reception desk (Building B, 1st floor)\n\nSome tests require special preparation like fasting - we'll inform you of any requirements when scheduling."
            },
            {
                "regex": r"(?:do I need|is it necessary to have|should I get) (?:a|an) (?:doctor's order|prescription|referral) (?:for|to get) (?:lab|laboratory|blood) (?:work|test|testing)",
                "response": lambda match, user: "Yes, most lab tests require a doctor's order. This ensures that appropriate tests are ordered and results are properly interpreted. Some basic wellness tests like cholesterol screening may be available without a prescription during our monthly health fairs. If you need testing but don't have a primary care provider, our urgent care clinic can evaluate you and order appropriate tests."
            },
            {
                "regex": r"(?:how long|how much time|when) (?:does it take|will it take) (?:to get|to receive|for) (?:lab|laboratory|test) results",
                "response": lambda match, user: "Lab result timeframes vary by test type:\n- Routine blood tests (CBC, metabolic panels): 24-48 hours\n- Basic urinalysis: Same day to 24 hours\n- Lipid panels: 24-48 hours\n- Thyroid function tests: 1-3 days\n- Cultures (bacterial, fungal): 3-5 days\n- Specialty or send-out tests: 5-10 business days\n\nResults are automatically posted to your patient portal once finalized and reviewed."
            },
            {
                "regex": r"(?:do I need to|should I|is it necessary to) (?:fast|avoid eating|stop eating) (?:before|prior to) (?:a|my|the) (?:lab|laboratory|blood) (?:work|test|testing)",
                "response": lambda match, user: "Fasting requirements depend on the specific tests ordered. Common tests requiring 8-12 hours of fasting include:\n- Lipid/cholesterol panels\n- Glucose tests\n- Comprehensive metabolic panels\n- Iron studies\n\nWater is permitted and usually encouraged during fasting. Morning appointments are recommended for fasting tests. If you're unsure about fasting requirements for your specific test, call our lab at (555) 345-6789."
            },
            {
                "regex": r"(?:how|where) (?:can|do) (?:I|you) (?:access|view|get|check|see) (?:my|the) (?:lab|laboratory|test) results",
                "response": lambda match, user: "You can access your lab results through several methods:\n1. Our patient portal (hospital.example.com/portal) - results post automatically once reviewed\n2. Call your ordering physician's office\n3. Request printed copies from our medical records department\n4. During your follow-up appointment with your doctor\n\nFor portal access issues, contact our helpdesk at (555) 444-3333. Please note that some sensitive test results may be released only after your doctor reviews them with you."
            },
            {
                "regex": r"(?:what|which) (?:lab|laboratory|blood) tests (?:do you|does the hospital|are) (?:offer|provide|perform|available)",
                "response": lambda match, user: "Our hospital laboratory offers comprehensive testing services including:\n\n• Hematology (complete blood counts, coagulation)\n• Chemistry (metabolic panels, lipids, cardiac markers)\n• Microbiology (cultures, sensitivity testing)\n• Immunology (antibody testing)\n• Urinalysis and toxicology\n• Endocrinology (hormone testing)\n• Molecular diagnostics and genetic testing\n• Pathology services\n\nFor specific test availability, please call our laboratory at (555) 345-6789."
            },
            {
                "regex": r"(?:how much|what) (?:does|is the cost of|will it cost for) (?:a|an) ([a-zA-Z\s]+) (?:test|lab|laboratory test)(?:\?)?",
                "response": lambda match, user: self.get_lab_test_cost(match.group(1))
            },
            {
                "regex": r"(?:do you|does the lab|does the hospital) (?:have|offer|provide) (?:walk-in|same day|immediate|no appointment) (?:lab|laboratory|blood) (?:testing|service)",
                "response": lambda match, user: "Yes, we offer walk-in laboratory services for patients with a doctor's order. Walk-in hours are:\n- Monday-Friday: 7:00 AM to 7:00 PM\n- Saturday: 8:00 AM to 3:00 PM\n- Sunday: Closed (except for inpatient and emergency services)\n\nWhile appointments aren't required, scheduled patients receive priority during busy periods. Wait times for walk-ins are typically 15-45 minutes depending on the time of day and test complexity."
            },
            {
                "regex": r"(?:how|what) (?:should I|do I need to) prepare (?:for|before) (?:a|my|the) ([a-zA-Z\s]+) (?:test|lab test)",
                "response": lambda match, user: self.get_lab_test_preparation(match.group(1))
            },
            {
                "regex": r"(?:can|will) (?:my|the) (?:primary care|referring|outside) doctor (?:receive|get|access) (?:my|the) (?:lab|laboratory|test) results",
                "response": lambda match, user: "Yes, your primary care or referring physician will receive your lab results if they ordered the tests or were listed as a copy recipient. Results are typically sent via secure electronic transmission, fax, or mail depending on the provider's preference. If you see providers outside our hospital system, you may need to complete a release form to have results sent to them. This can be done at our medical records department or through your patient portal."
            },
            # Add 90+ more lab test patterns here
        ]

    def get_emergency_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:symptoms|conditions|situations) (?:should|would) (?:I|someone) (?:go|come|visit) (?:to|the) (?:ER|emergency room|emergency department) (?:for)",
                "response": lambda match, user: "You should go to the ER for potentially life-threatening conditions including:\n• Chest pain or pressure\n• Difficulty breathing\n• Severe bleeding or trauma\n• Sudden numbness/weakness (stroke symptoms)\n• Severe abdominal pain\n• Serious burns\n• Head injuries with loss of consciousness\n• Seizures\n• High fever with headache and stiff neck\n• Poisoning or overdose\n\nFor less severe issues, urgent care may be more appropriate and have shorter wait times."
            },
            {
                "regex": r"(?:what|where) is (?:the|your) (?:ER|emergency room|emergency department) (?:location|situated|address)",
                "response": lambda match, user: "Our Emergency Department is located at the south entrance of the main hospital campus at 123 Medical Center Drive. It's accessible via Emergency Drive with dedicated parking directly outside the entrance. The ER entrance is clearly marked with red 'EMERGENCY' signs and is separate from the main hospital entrance. Ambulances arrive at a dedicated ambulance bay adjacent to the walk-in entrance."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:ER|emergency room|emergency department) (?:hours|timing|times)",
                "response": lambda match, user: "Our Emergency Department is open 24 hours a day, 7 days a week, 365 days a year, including all holidays. We never close and are always staffed with emergency medicine physicians, nurses, and support personnel ready to handle any medical emergency."
            },
            {
                "regex": r"(?:how long|what) (?:is|are) (?:the|typical|average|current) (?:ER|emergency room|emergency department) (?:wait times|waiting period)",
                "response": lambda match, user: "Current ER wait times vary based on patient volume and the severity of cases. Patients are seen based on medical urgency, not arrival time. Life-threatening conditions receive immediate attention, while less severe issues may wait longer. Our current estimated wait time for non-critical cases is typically 30-120 minutes. You can check our real-time ER wait times on our website or by calling (555) 911-7000."
            },
            {
                "regex": r"(?:what|which) (?:documents|items|things) (?:should|do) (?:I|you) (?:bring|take|have) (?:to|for) (?:the|an) (?:ER|emergency room|emergency) (?:visit)",
                "response": lambda match, user: "If possible, bring these items to the ER:\n1. Photo ID\n2. Insurance card\n3. List of current medications (or the medication bottles)\n4. List of allergies\n5. Medical history summary\n6. Contact information for your primary doctor\n7. Advanced directives if applicable\n\nIn a life-threatening emergency, don't delay coming to the ER if you don't have these items - your health comes first, and we can collect this information later."
            },
            {
                "regex": r"(?:what|how) (?:is|happens|works) (?:the|your) (?:ER|emergency room|emergency department) (?:triage|sorting|prioritization) (?:process|system|procedure)",
                "response": lambda match, user: "Our ER triage process works as follows:\n1. Initial assessment by a triage nurse upon arrival\n2. Vital signs and chief complaint recorded\n3. Assignment of a triage level (1-5) based on medical urgency\n4. Level 1 (most critical) patients seen immediately\n5. Other patients seen in order of medical necessity, not arrival time\n\nThis ensures that life-threatening conditions receive immediate attention while providing care to all patients as efficiently as possible."
            },
            {
                "regex": r"(?:can|will) (?:family members|relatives|visitors) (?:stay|remain|be) (?:with|alongside) (?:me|patients|a patient) (?:in|during) (?:the|an) (?:ER|emergency room|emergency) (?:visit)",
                "response": lambda match, user: "Generally, one family member or support person may accompany a patient in the ER treatment area, with some limitations:\n- For pediatric patients, one or both parents/guardians are welcome\n- During certain procedures or in critical situations, visitors may be asked to step out temporarily\n- In trauma or resuscitation cases, family may be directed to a dedicated waiting area\n- During high census or pandemic restrictions, visitor policies may be modified\n\nOur goal is to support patients while maintaining a safe care environment."
            },
            {
                "regex": r"(?:what is|how much is|how much does) (?:the|an) (?:ER|emergency room|emergency department) (?:visit|trip) (?:cost|charge)",
                "response": lambda match, user: "Emergency room costs vary widely depending on the level of care required, tests performed, and treatments provided. Base ER visit charges typically range from $500-$3,000 before insurance. Additional costs may include:\n- Imaging (X-rays, CT scans, MRIs)\n- Laboratory tests\n- Medications\n- Specialist consultations\n- Procedures\n\nYour actual out-of-pocket cost depends on your insurance coverage. Our financial counselors can help discuss payment options and financial assistance programs."
            },
            {
                "regex": r"(?:does|is) (?:the|your) (?:ER|emergency room|emergency department) (?:treat|handle|manage) (?:children|pediatric|kids) (?:emergencies|cases|patients)",
                "response": lambda match, user: "Yes, our Emergency Department is equipped to handle pediatric emergencies. We have dedicated pediatric emergency physicians on staff 24/7, specialized pediatric nurses, child-sized equipment, and child-friendly treatment rooms. For extremely critical pediatric cases, we have transfer protocols in place with the regional children's hospital if specialized inpatient care is needed. Our goal is to make emergency care as comfortable and non-threatening as possible for our youngest patients."
            },
            {
                "regex": r"(?:what|how) (?:happens|occurs|is the process) (?:after|following) (?:an|the) (?:ER|emergency room|emergency department) (?:visit|treatment)",
                "response": lambda match, user: "After ER treatment, several things may happen:\n1. Discharge home with follow-up instructions and possibly prescriptions\n2. Admission to the hospital if ongoing care is needed\n3. Transfer to another facility for specialized care\n4. Scheduling of follow-up appointments\n\nYou'll receive discharge instructions explaining your diagnosis, treatment, medication instructions, and when to seek further care. A summary will be sent to your primary care provider, and you can access your visit details through our patient portal."
            },
            # Add 90+ more emergency patterns here
        ]

    def get_admission_patterns(self):
        return [
            {
                "regex": r"(?:what|which|how) (?:is|are) (?:the|your) (?:process|procedure|steps) (?:for|to) (?:patient|hospital) admission",
                "response": lambda match, user: "Our hospital admission process includes:\n1. Admission order from your doctor (planned) or ER physician (emergency)\n2. Registration with patient information and insurance verification\n3. Consent forms and HIPAA acknowledgment\n4. Financial counseling if needed\n5. Medical history and medication reconciliation\n6. Vital signs and initial nursing assessment\n7. Assignment to appropriate unit based on medical needs\n8. Transport to hospital room\n\nFor planned admissions, we recommend pre-registration by calling (555) 789-0123 at least 48 hours in advance."
            },
            {
                "regex": r"(?:what|which) (?:documents|items|paperwork|things) (?:do I need|should I bring|are required) (?:for|during) (?:hospital|patient) admission",
                "response": lambda match, user: "Please bring these items for hospital admission:\n• Photo ID\n• All insurance cards\n• List of current medications and dosages\n• Advance directives (living will, healthcare power of attorney)\n• Contact information for your primary doctor\n• Medical history information\n• Any relevant medical records or test results\n• Payment method for potential copays/deductibles\n• Personal items for your stay (limited clothing, toiletries)\n• CPAP or other home medical devices you use regularly"
            },
            {
                "regex": r"(?:can|do) (?:I|you|patients) (?:request|choose|select) (?:a|my|the) (?:room|room type|accommodation) (?:during|for) (?:admission|hospital stay)",
                "response": lambda match, user: "We offer several room types subject to availability and medical necessity:\n1. Standard semi-private rooms (two patients)\n2. Private rooms (available with physician order for isolation or medical necessity)\n3. Deluxe private rooms (additional daily fee applies)\n\nRoom assignments prioritize medical needs first. While we try to accommodate preferences, availability varies by department and hospital census. You can request a specific room type during pre-registration, but cannot guarantee fulfillment of all requests."
            },
            {
                "regex": r"(?:how|what time|when) (?:early|soon|in advance) (?:should|do) (?:I|patients) (?:arrive|come|check in) (?:for|before) (?:a|my|planned|scheduled) admission",
                "response": lambda match, user: "For planned hospital admissions, please arrive 2 hours before your scheduled admission time to complete necessary paperwork and pre-admission testing. For surgical admissions, arrival times are typically 2-3 hours before your scheduled procedure and will be specified in your pre-surgical instructions. For maternity admissions in active labor, come directly to the Labor & Delivery department entrance."
            },
            {
                "regex": r"(?:what|which) (?:personal|private) (?:items|belongings|things) (?:can|should|may) (?:I|patients) (?:bring|pack|have) (?:for|during) (?:a|my|the) (?:hospital|inpatient) stay",
                "response": lambda match, user: "Recommended personal items for your hospital stay:\n• Comfortable pajamas or loose clothing\n• Non-slip slippers or footwear\n• Basic toiletries (toothbrush, toothpaste, etc.)\n• Eyeglasses, hearing aids, dentures if used\n• Cell phone and charger\n• Small amount of cash for incidentals\n• Book or tablet for entertainment\n\nPlease leave valuables, jewelry, and large sums of money at home. The hospital provides basic toiletries if needed, but personal items often improve comfort during your stay."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:hospital|facility) (?:visiting|visitor) (?:hours|policy|rules)",
                "response": lambda match, user: "Our general visiting hours are 8:00 AM to 8:00 PM daily. Specialized units have modified policies:\n• ICU: Limited to 15-minute visits at specific times (10 AM, 2 PM, 6 PM)\n• Maternity: Father/partner can visit 24/7; others during regular hours\n• Pediatrics: Parents/guardians can stay 24/7; others during regular hours\n• Behavioral Health: 6:00-8:00 PM weekdays, 2:00-4:00 PM weekends\n\nAll visitors must check in at the information desk for a visitor badge. Children under 12 must be accompanied by an adult."
            },
            {
                "regex": r"(?:can|are|is it allowed for) (?:family members|relatives|someone) (?:stay|sleep|remain) (?:overnight|during the night|after hours) (?:with|alongside) (?:patients|me)",
                "response": lambda match, user: "Overnight stays for family members are allowed in these situations:\n1. Parents/guardians of pediatric patients (under 18)\n2. Partners of maternity patients\n3. Family of end-of-life patients\n4. Caregivers of patients with special needs (dementia, disability)\n\nOne adult family member can stay overnight in these cases. A reclining chair or sleeper sofa is provided. Please inform the nursing staff if someone will be staying so they can provide linens and make accommodations."
            },
            {
                "regex": r"(?:how long|what) (?:is|will be|can I expect) (?:the|my|typical|average) (?:hospital|inpatient) stay (?:for|after) ([a-zA-Z\s]+)",
                "response": lambda match, user: self.get_average_stay_duration(match.group(1))
            },
            {
                "regex": r"(?:what|which) (?:meals|food|dining options) (?:are|is) (?:provided|available|offered) (?:for|to|during) (?:inpatients|admitted patients|patients)",
                "response": lambda match, user: "Inpatients receive three meals daily according to physician-ordered diet plans. Breakfast is served 7:00-8:30 AM, lunch 11:30 AM-1:00 PM, and dinner 5:00-6:30 PM. Our room service program allows patients to order from a menu tailored to their dietary restrictions between 7:00 AM and 7:00 PM by calling ext. 3663 (FOOD) from your room phone. Special diets (diabetic, cardiac, renal, gluten-free, etc.) are accommodated with dietitian consultation."
            },
            {
                "regex": r"(?:what|which) (?:medical professionals|healthcare team|staff) (?:will|can) (?:I|patients) (?:see|meet|interact with) (?:during|throughout) (?:my|the|a) (?:hospital|inpatient) stay",
                "response": lambda match, user: "During your hospital stay, you may interact with various healthcare professionals:\n• Attending physician (oversees your care)\n• Hospitalist (manages inpatient treatment)\n• Specialist consultants as needed\n• Resident physicians and medical students (teaching hospital)\n• Nurses (primary caregivers, present 24/7)\n• Nursing assistants (help with daily care needs)\n• Therapists (physical, occupational, respiratory, speech)\n• Dietitians\n• Case managers/social workers\n• Phlebotomists (blood draws)\n• Technicians (EKG, imaging, etc.)\n\nAll staff should introduce themselves and wear identification badges."
            },
            # Add 90+ more admission patterns here
        ]

    def get_discharge_patterns(self):
        return [
            {
                "regex": r"(?:what|which|how) (?:is|are) (?:the|your) (?:process|procedure|steps) (?:for|to) (?:patient|hospital) discharge",
                "response": lambda match, user: "Our discharge process includes:\n1. Discharge order from your physician\n2. Discharge planning meeting with nurse and/or case manager\n3. Education on home care, medications, and follow-up\n4. Prescription preparation by pharmacy\n5. Transportation arrangements if needed\n6. Final paperwork and instructions\n7. Escort to hospital exit\n\nDischarges typically occur between 11:00 AM and 2:00 PM. The process takes 1-2 hours after your doctor writes the discharge order."
            },
            {
                "regex": r"(?:what|which) (?:information|documents|paperwork|instructions) (?:will|do|should) (?:I|patients) (?:receive|get|be given) (?:at|during|upon) discharge",
                "response": lambda match, user: "At discharge, you'll receive a comprehensive packet including:\n• Written discharge instructions with care guidelines\n• Medication list with dosages and schedules\n• New prescription information\n• Follow-up appointment details\n• Signs/symptoms requiring medical attention\n• Dietary and activity restrictions\n• Wound care instructions if applicable\n• Contact numbers for questions\n• Educational materials about your condition\n\nYour nurse will review all information with you before you leave. Please ask questions if anything is unclear."
            },
            {
                "regex": r"(?:how|what) (?:do|can|should) (?:I|patients) (?:arrange|get|organize) (?:transportation|a ride|travel) (?:after|following|for|upon) discharge",
                "response": lambda match, user: "Transportation options after discharge include:\n1. Family member or friend pickup (recommended)\n2. Hospital-arranged transportation for eligible patients (needs case manager approval)\n3. Taxi or rideshare services (Uber/Lyft)\n4. Medical transport services for those requiring assistance\n5. Public transportation if appropriate for your condition\n\nPlease arrange transportation before discharge day. For assistance with special transportation needs, speak with your case manager or social worker at least 24 hours before discharge."
            },
            {
                "regex": r"(?:what|which|how) (?:follow-up|followup) (?:appointments|care|visits) (?:will|should|do) (?:I|patients) (?:need|have|require) (?:after|following) discharge",
                "response": lambda match, user: "Follow-up care after discharge varies by condition, but typically includes:\n1. Primary care visit within 1-2 weeks\n2. Specialist follow-up as recommended by your physician\n3. Therapy appointments if prescribed (physical, occupational, speech)\n4. Home health visits for eligible patients\n\nYour discharge planner will help schedule initial follow-up appointments before you leave. These appointments are crucial for monitoring your recovery and preventing readmission. If you need to reschedule, please call the provider's office directly."
            },
            {
                "regex": r"(?:what|which) (?:medications|prescriptions|drugs) (?:will|should|do) (?:I|patients) (?:take|need|receive) (?:after|upon|following) discharge",
                "response": lambda match, user: "Your discharge medications will be detailed in your discharge instructions and may include:\n1. New medications prescribed during hospitalization\n2. Continued home medications (possibly with adjusted dosages)\n3. Temporary medications for recovery (pain management, antibiotics)\n\nYour physician will review medication changes. Our discharge pharmacist can fill prescriptions before you leave if desired. We'll provide a complete medication list, including purpose, dosage, schedule, and potential side effects. Please inform us of any medication concerns or financial constraints."
            },
            {
                "regex": r"(?:what|which) (?:diet|nutrition|food|eating) (?:restrictions|guidelines|recommendations) (?:should|will|do) (?:I|patients) (?:follow|have) (?:after|upon|following) discharge",
                "response": lambda match, user: "Dietary guidelines after discharge depend on your specific condition. Your discharge instructions will include detailed nutrition recommendations. Common dietary plans include:\n- Cardiac diet (low sodium, low fat)\n- Diabetic diet (carbohydrate controlled)\n- Renal diet (restricted sodium, potassium, phosphorus)\n- Soft or modified texture diets following certain surgeries\n- Progressive diets for GI conditions\n\nA registered dietitian may consult with you before discharge for personalized guidance. Follow-up with a dietitian can be arranged if needed."
            },
            {
                "regex": r"(?:what|which) (?:activity|exercise|physical) (?:restrictions|limitations|guidelines) (?:should|will|do) (?:I|patients) (?:have|follow) (?:after|upon|following) discharge",
                "response": lambda match, user: "Activity guidelines after discharge vary by condition and procedure. Your discharge instructions will specify limitations such as:\n- Weight-bearing restrictions after orthopedic procedures\n- Lifting limitations after abdominal or chest surgery\n- Driving restrictions while on certain medications\n- Gradual return to normal activities timeline\n- Specific exercises recommended for recovery\n\nFollowing these guidelines is crucial for proper healing and preventing complications. A physical therapist may provide specific exercises. Contact your doctor before resuming activities not addressed in your instructions."
            },
            {
                "regex": r"(?:what|which) (?:warning signs|red flags|symptoms|complications) (?:should|would|do) (?:I|patients) (?:watch for|look for|monitor) (?:after|following) discharge",
                "response": lambda match, user: "After discharge, contact your doctor or return to the ER if you experience:\n• Fever above 101°F (38.3°C)\n• Uncontrolled pain not relieved by prescribed medications\n• Increasing redness, warmth, pain, or drainage from surgical sites\n• New or worsening shortness of breath\n• Chest pain\n• Severe headache or confusion\n• Persistent nausea/vomiting or inability to keep fluids down\n• Signs of infection (increased pain, swelling, redness, warmth, discharge)\n• Blood in urine, stool, or vomit\n• Falls or new injuries\n\nYour discharge instructions will include condition-specific warning signs to monitor."
            },
            {
                "regex": r"(?:how|what) (?:do|can|should) (?:I|patients) (?:manage|handle|control) (?:pain|discomfort) (?:at|after|following|upon) discharge",
                "response": lambda match, user: "Pain management after discharge may include:\n1. Prescribed pain medications (take exactly as directed)\n2. Over-the-counter medications if recommended\n3. Non-pharmaceutical approaches:\n   - Cold/heat therapy as directed\n   - Proper positioning and body mechanics\n   - Gentle activity as approved by your doctor\n   - Relaxation techniques\n\nTrack your pain levels on a scale of 0-10. Contact your healthcare provider if pain is not controlled, worsens suddenly, or changes in character. Never take more than the prescribed dose of pain medication."
            },
            {
                "regex": r"(?:what|which) (?:home health|nursing|care) (?:services|assistance|support) (?:are|is|might be) (?:available|offered|provided) (?:after|following|upon) discharge",
                "response": lambda match, user: "Home health services available after discharge may include:\n1. Skilled nursing visits (wound care, medication management, assessments)\n2. Physical, occupational, and speech therapy at home\n3. Medical social worker services\n4. Home health aide assistance with personal care\n5. Telehealth monitoring for certain conditions\n\nEligibility is determined by your insurance and medical necessity documented by your physician. Your case manager will arrange these services before discharge if needed. Most insurance plans, including Medicare, cover home health when medically necessary."
            },
            # Add 90+ more discharge patterns here
        ]

    def get_insurance_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:insurance|insurance plans|insurance providers) (?:do you|does the hospital) (?:accept|take|work with)",
                "response": lambda match, user: "We accept most major insurance plans including:\n• Blue Cross Blue Shield (all plans)\n• Aetna\n• Cigna\n• UnitedHealthcare\n• Medicare and Medicare Advantage plans\n• Medicaid and Medicaid managed care\n• Tricare\n• Humana\n• Kaiser Permanente\n\nFor verification of your specific plan coverage, please contact our insurance verification department at (555) 456-7890 with your policy information before your visit."
            },
            {
                "regex": r"(?:how|what) (?:do|can|should) (?:I|patients) (?:verify|check|confirm) (?:if|that|whether) (?:my|their) insurance (?:covers|will cover|is accepted)",
                "response": lambda match, user: "To verify your insurance coverage:\n1. Call our insurance verification team at (555) 456-7890\n2. Provide your insurance ID, group number, and plan details\n3. Specify the service or procedure you're seeking\n\nAlternatively, contact your insurance provider directly using the member services number on your insurance card. For planned procedures, we recommend verification at least 3-5 business days in advance to allow time to address any coverage issues."
            },
            {
                "regex": r"(?:what|which) (?:information|documents|details) (?:do|should) (?:I|patients) (?:need|bring|provide) (?:for|about|regarding) insurance (?:verification|coverage)",
                "response": lambda match, user: "For insurance verification, please provide:\n• Insurance ID card (front and back)\n• Photo identification\n• Name of policyholder if not yourself\n• Group and policy numbers\n• Insurance company phone number\n• Secondary insurance information if applicable\n• Referral or authorization numbers if required by your plan\n• Worker's compensation information if applicable\n\nIt's best to bring physical copies of your insurance cards to every hospital visit."
            },
            {
                "regex": r"(?:what|how) (?:if|about|happens when) (?:I don't have|I have no|there is no) (?:insurance|health insurance|medical insurance|coverage)",
                "response": lambda match, user: "If you don't have insurance, we offer several options:\n1. Self-pay discounts of 30-40% for paying at time of service\n2. Financial assistance programs based on income (may provide free or reduced-cost care)\n3. Payment plans without interest\n4. Assistance applying for Medicaid or other government programs\n5. Information about healthcare marketplace options\n\nOur financial counselors can help determine your eligibility for these programs. Please contact them at (555) 789-3456 to discuss your specific situation."
            },
            {
                "regex": r"(?:how|what) (?:do|does) (?:pre-authorization|prior authorization|pre-approval) (?:work|process work|system work)",
                "response": lambda match, user: "The pre-authorization process works as follows:\n1. Your doctor orders a procedure, test, or medication requiring authorization\n2. Our authorization team submits clinical information to your insurance\n3. Insurance reviews the request (typically takes 3-10 business days)\n4. Insurance provides approval or denial\n5. Our team notifies you and your provider of the decision\n\nSome services commonly requiring pre-authorization include:\n- MRI/CT scans\n- Surgeries\n- Specialty medications\n- Inpatient admissions\n- Certain therapy services\n\nAlways check if your procedure needs pre-authorization before scheduling."
            },
            {
                "regex": r"(?:what|which) (?:services|procedures|treatments) (?:require|need) (?:pre-authorization|prior authorization|pre-approval)",
                "response": lambda match, user: "Services typically requiring pre-authorization include:\n• Advanced imaging (MRI, CT, PET scans)\n• Surgeries and outpatient procedures\n• Hospital admissions (except emergencies)\n• Specialty medications and infusions\n• Durable medical equipment\n• Home health services\n• Physical/occupational/speech therapy beyond initial evaluation\n• Sleep studies\n• Genetic testing\n• Transplant services\n\nRequirements vary by insurance plan. Our scheduling team will help determine if your service needs authorization when you make your appointment."
            },
            {
                "regex": r"(?:what|which) (?:is|about|happens with) (?:out-of-network|non-participating|non-par) (?:coverage|benefits|insurance)",
                "response": lambda match, user: "For out-of-network care:\n1. Your out-of-pocket costs will typically be higher\n2. You may need to pay in full at time of service and submit claims yourself\n3. Your insurance may cover a lower percentage of charges\n4. You may have a separate, higher deductible for out-of-network care\n\nIn some situations (emergencies, specialized care unavailable in-network), you can request an in-network exception from your insurance. Our financial counselors can help explain your estimated costs for out-of-network services at (555) 789-3456."
            },
            {
                "regex": r"(?:how|what) (?:do|does|about) (?:co-pays|copays|co-payments) (?:work|process work|collection work)",
                "response": lambda match, user: "Copays work as follows:\n1. Determined by your specific insurance plan\n2. Usually collected at time of service\n3. May differ by service type (office visit, specialist, ER, etc.)\n4. Separate from deductibles and coinsurance\n\nWe accept copayments via cash, check, credit/debit cards, and HSA/FSA cards. For hospital stays, copays are typically collected upon admission. You can check your expected copay amount by calling our financial services team at (555) 789-3456 with your insurance information."
            },
            {
                "regex": r"(?:what|which|how) (?:is|are|about) (?:my|patient) (?:financial responsibility|out-of-pocket costs|payment obligations)",
                "response": lambda match, user: "Your financial responsibility typically includes:\n1. Copays (fixed amount per visit/service)\n2. Deductible (amount you pay before insurance starts covering)\n3. Coinsurance (percentage of costs after deductible is met)\n4. Non-covered services (items your plan doesn't cover)\n\nThe specific amounts depend on your insurance plan terms. Before scheduled procedures, we offer cost estimates based on your insurance benefits. For a personalized estimate, contact our financial counselors at (555) 789-3456 with your insurance information and planned procedure codes."
            },
            {
                "regex": r"(?:how|what) (?:do|can|about) (?:insurance claims|medical claims|claim submission) (?:work|process work|processing work)",
                "response": lambda match, user: "Our insurance claims process works as follows:\n1. After your visit, we submit claims electronically to your insurance within 2-5 business days\n2. Insurance typically processes claims in 14-30 days\n3. We receive an Explanation of Benefits (EOB) detailing covered services\n4. Your responsibility is calculated based on your insurance terms\n5. You receive a statement for any remaining balance\n\nIf your claim is denied, our billing team works to appeal or resubmit as needed. For questions about a specific claim, contact our billing department at (555) 987-6543 with your account number and service date."
            },
            # Add 90+ more insurance patterns here
        ]

    def get_visitor_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:visitor|visitation|visiting) (?:hours|times|policy)",
                "response": lambda match, user: "Our general visiting hours are 8:00 AM to 8:00 PM daily. However, specific units have different policies:\n\n• ICU: 10:00-11:00 AM, 1:00-2:00 PM, 4:00-5:00 PM, 8:00-9:00 PM (15 minutes per visit)\n• Maternity: 9:00 AM to 9:00 PM (partners may stay 24/7)\n• Pediatrics: Parents/guardians 24/7, others 10:00 AM to 8:00 PM\n• Behavioral Health: 6:00-8:00 PM weekdays, 2:00-4:00 PM weekends\n• Emergency Department: Limited to 1-2 visitors per patient based on department capacity\n\nVisiting hours may be adjusted during public health emergencies."
            },
            {
                "regex": r"(?:how many|what number of) (?:visitors|people|family members) (?:can|are allowed|may) (?:visit|see|be with) (?:a patient|patients|someone) (?:at|during) (?:one|a|the same) time",
                "response": lambda match, user: "Visitor limits vary by unit:\n• General medical/surgical floors: 2 visitors at a time\n• ICU: 2 visitors during designated hours\n• Maternity: 2 visitors plus partner/support person\n• Pediatrics: 2 parents/guardians at all times, 1-2 additional visitors during regular hours\n• End-of-life care: Special arrangements for additional family members\n• Isolation rooms: May have additional restrictions\n\nCharge nurses may make exceptions for special circumstances. These limits help maintain a healing environment and allow our staff to provide optimal care."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:rules|policy|guidelines) (?:for|about|regarding) (?:children|kids|minors) (?:visitors|visiting)",
                "response": lambda match, user: "Our policy for child visitors:\n• Children under 14 must be accompanied by an adult who is not the patient\n• Children should be supervised at all times\n• Children are not permitted in isolation rooms except in special circumstances\n• Children must be free from illness/infection (no fever, cough, rash, etc.)\n• Some units (ICU, oncology) may have additional age restrictions\n• Children visiting siblings in maternity/pediatrics may require special screening\n\nWe recognize the importance of family connections but must balance this with infection control and patient care needs."
            },
            {
                "regex": r"(?:do|are) (?:visitors|family members|friends) (?:need|required) (?:to|must) (?:check in|register|sign in) (?:somewhere|anywhere|at a desk)",
                "response": lambda match, user: "Yes, all visitors must check in at these locations:\n1. Main entrance information desk (5:00 AM - 9:00 PM)\n2. Emergency Department entrance (after hours)\n3. Unit-specific nursing stations outside regular hours\n\nVisitors will need to:\n• Present photo ID\n• Receive a visitor badge\n• Sign in with name, contact information, and patient being visited\n• Complete a brief health screening questionnaire\n\nThis process helps us maintain campus security and assist with contact tracing when necessary."
            },
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors) (?:find|locate|get to) (?:a patient's|someone's|the) (?:room|location)",
                "response": lambda match, user: "To locate a patient's room:\n1. Visit the information desk in the main lobby with the patient's full name\n2. Use our electronic directory kiosks (located at all major entrances)\n3. Call our patient information line at (555) 123-4500\n\nFor privacy reasons, we can only provide room information if the patient has agreed to be listed in our directory. Some patients (behavioral health, certain VIPs, etc.) may opt out of the directory. Staff will confirm the patient's privacy settings before providing location information."
            },
            {
                "regex": r"(?:what|which) (?:items|things|gifts) (?:can|are allowed to|may) (?:I|visitors) (?:bring|take|deliver) (?:to|for) (?:patients|a patient|someone)",
                "response": lambda match, user: "Acceptable items to bring patients include:\n• Clothing, pajamas, slippers (non-slip)\n• Toiletries without alcohol content\n• Books, magazines, electronic devices\n• Eyeglasses, hearing aids, dentures\n• Photos and small mementos\n• Sealed, commercially prepared food (with nursing approval)\n• Flowers (except in ICU, oncology, transplant units)\n\nProhibited items include:\n• Alcohol, drugs, tobacco, e-cigarettes\n• Weapons of any kind\n• Homemade food in most cases\n• Latex balloons (mylar are acceptable)\n• Strong perfumes/colognes\n• Valuable items"
            },
            {
                "regex": r"(?:are there|do you have|is there) (?:any|special) (?:overnight|after hours) (?:accommodations|arrangements|options) (?:for|available to) (?:visitors|family|relatives)",
                "response": lambda match, user: "Overnight accommodations for visitors include:\n• In-room sleep chairs in pediatric units for parents/guardians\n• Maternity unit accommodations for one support person\n• Family lounges with recliners in some units for family of critically ill patients\n• Hospital-affiliated hotel with reduced rates (connected via skyway)\n• Ronald McDonald House for families of pediatric patients (based on eligibility)\n\nFor overnight stays in patient rooms, please obtain approval from the charge nurse. Accommodations are limited to one visitor per patient except in special circumstances."
            },
            {
                "regex": r"(?:what|where) (?:are|is) (?:the|your) (?:visitor|guest|family) (?:parking|parking options|places to park)",
                "response": lambda match, user: "Visitor parking options include:\n1. Main Visitor Garage (adjacent to hospital entrance)\n   - $2 per hour, $10 daily maximum\n   - Open 24/7\n2. Surface Lot B (across from Emergency Department)\n   - $1 per hour, $8 daily maximum\n   - Open 24/7\n3. Valet parking at main entrance\n   - $5 for up to 4 hours, $15 for 4+ hours\n   - Available 6:00 AM - 8:00 PM weekdays\n\nDiscount parking passes (5-day, 10-day, monthly) are available at the information desk for families of extended-stay patients."
            },
            {
                "regex": r"(?:are there|do you have|is there) (?:any|special) (?:restrictions|limitations|special rules) (?:for|on|about) (?:visitors|visiting) (?:in|on|at) (?:specific|certain|particular) (?:units|departments|areas)",
                "response": lambda match, user: "Special visitor restrictions by unit include:\n\n• ICU: Limited to immediate family, max 2 visitors during specific hours, 15-minute visits\n• Oncology: No visitors with recent illness or infection, no fresh flowers/plants\n• Transplant Units: Strict infection control, visitors screened for infections\n• Behavioral Health: Approved visitor list only, belongings checked at entrance\n• Labor & Delivery: Limited to 2-3 designated support persons throughout stay\n• NICU: Parents/guardians only, with special handwashing protocols\n• Isolation Rooms: PPE required, limited duration visits\n\nUnit-specific policies are posted at entrances to each department."
            },
            {
                "regex": r"(?:what|which) (?:amenities|facilities|services) (?:are|is) (?:available|offered|provided) (?:to|for) (?:visitors|family members|guests)",
                "response": lambda match, user: "Amenities for visitors include:\n• Cafeteria (1st floor, open 6:30 AM - 8:00 PM)\n• Coffee shop (main lobby, open 6:00 AM - 6:00 PM weekdays, 8:00 AM - 2:00 PM weekends)\n• Vending machines (various locations, 24/7)\n• Chapel/meditation room (2nd floor, always open)\n• Family lounges on each unit with TV and comfortable seating\n• Free Wi-Fi throughout facility\n• ATM in main lobby\n• Gift shop (main lobby, 9:00 AM - 7:00 PM weekdays, 11:00 AM - 5:00 PM weekends)\n• Business center with computers/printer (3rd floor)\n• Lactation rooms for nursing mothers\n• Shower facilities for long-term family members (request from nursing staff)"
            },
            # Add 90+ more visitor patterns here
        ]

    def get_cafeteria_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:cafeteria|dining|food service) (?:hours|timing|schedule)",
                "response": lambda match, user: "Our main cafeteria hours are:\n• Breakfast: 6:30 AM - 10:00 AM\n• Lunch: 11:00 AM - 2:00 PM\n• Dinner: 4:30 PM - 8:00 PM\n\nThe grab-and-go section with sandwiches, salads, and snacks is available from 6:30 AM to 8:00 PM daily. The coffee bar operates from 6:30 AM to 6:00 PM on weekdays and 7:00 AM to 2:00 PM on weekends. Vending machines with snacks and beverages are available 24/7 in the vending area near the cafeteria and in waiting areas throughout the hospital."
            },
            {
                "regex": r"(?:where|what) (?:is|are) (?:the|your) (?:cafeteria|dining area|food court) (?:located|situated)",
                "response": lambda match, user: "Our main cafeteria is located on the first floor of the South Wing (Building B). From the main entrance, follow the blue line on the floor or the overhead signs toward 'Dining Services.' The cafeteria is near elevator bank B, across from the gift shop. Additional dining options include:\n• Coffee shop in the main lobby\n• Doctors' dining room on the second floor (physicians only)\n• Courtyard Café in the Medical Office Building (Building C, first floor)\n• Rooftop Garden Café in the Cancer Center (seasonal, May-October)"
            },
            {
                "regex": r"(?:what|which) (?:food|meal|dining) (?:options|choices|selections) (?:are|is) (?:available|offered|served) (?:in|at) (?:the|your) cafeteria",
                "response": lambda match, user: "Our cafeteria offers diverse options including:\n• Hot entrée station with daily specials (including vegetarian option)\n• Grill serving burgers, sandwiches, and breakfast items\n• Made-to-order deli sandwich station\n• Pizza and pasta bar\n• Fresh salad bar with over 30 items\n• Sushi station (Tuesday and Thursday only)\n• Soup station (3-4 daily selections, including vegetarian)\n• International cuisine station (rotating menu)\n• Grab-and-go section with pre-made options\n• Dessert and bakery section\n• Beverage station\n\nWeekly menus are posted on our hospital website and at cafeteria entrances."
            },
            {
                "regex": r"(?:what|which|how much) (?:are|is) (?:the|your) (?:cafeteria|food|meal) (?:prices|cost|rates)",
                "response": lambda match, user: "Our cafeteria prices are designed to be affordable:\n• Breakfast items: $1.50-$5.50\n• Sandwiches and burgers: $4.00-$7.00\n• Hot entrées: $5.50-$8.50\n• Pizza slices: $2.50-$3.50\n• Salad bar: $0.59/ounce\n• Soups: $2.50 (cup), $3.75 (bowl)\n• Side dishes: $1.50-$3.00\n• Desserts: $1.50-$4.00\n• Beverages: $1.50-$3.00\n\nHospital employees receive a 20% discount with ID badge. Weekly specials and combo meals offer additional savings. We accept cash, credit/debit cards, and mobile payment options."
            },
            {
                "regex": r"(?:do you|does the cafeteria) (?:have|offer|accommodate) (?:special|specific|specialized) (?:diets|dietary needs|food restrictions)",
                "response": lambda match, user: "Yes, our cafeteria accommodates various dietary needs:\n• Vegetarian and vegan options daily at all stations\n• Gluten-free selections (clearly labeled)\n• Kosher pre-packaged meals available upon request\n• Halal options available certain days (check weekly menu)\n• Low-sodium choices (identified with heart-healthy symbols)\n• Diabetic-friendly options (carb counts displayed)\n• Allergen information posted for all prepared foods\n\nOur dietitians work with food services to ensure diverse dietary needs are met. For specific dietary concerns, staff at the cafeteria information desk can provide detailed ingredient information."
            },
            {
                "regex": r"(?:can|do) (?:visitors|family members|guests) (?:eat|dine) (?:in|at) (?:the|your) cafeteria",
                "response": lambda match, user: "Yes, visitors are welcome to eat in our cafeteria. The facility is open to patients (who are medically able to leave their rooms), visitors, staff, and the general public. The cafeteria provides a comfortable dining environment with various seating options including:\n• Main dining area with tables for 2-6 people\n• Private booths along the windows\n• Outdoor patio seating (seasonal)\n• Quiet section for those seeking a more peaceful environment\n\nVisitors can also purchase food to take back to patient waiting areas or patient rooms if appropriate for the patient's dietary restrictions."
            },
            {
                "regex": r"(?:can|do) (?:I|visitors|family) (?:bring|take|deliver) (?:cafeteria|outside) food (?:to|for) (?:patients|a patient)",
                "response": lambda match, user: "Visitors may bring food from our cafeteria to patients, but with important considerations:\n1. Check with the nursing staff first to ensure the patient doesn't have dietary restrictions\n2. Many patients are on specific diets (low-sodium, diabetic, liquid, etc.) prescribed by their physicians\n3. Some medications interact with certain foods\n4. Some units (ICU, pre/post-surgical) may restrict outside food\n\nIf approved by nursing staff, we provide disposable trays for transporting cafeteria food to patient rooms. Please note that homemade food from outside the hospital may have additional restrictions due to food safety protocols."
            },
            {
                "regex": r"(?:are there|do you have|is there) (?:any|other) (?:food|dining|meal) (?:options|alternatives|choices) (?:besides|other than|apart from) (?:the|your) cafeteria",
                "response": lambda match, user: "Yes, additional food options beyond our main cafeteria include:\n• Sunrise Coffee Shop (main lobby): specialty coffees, teas, pastries, sandwiches\n• Vending machines: throughout hospital, 24/7 access, offering snacks and beverages\n• Courtyard Café (Medical Office Building): lighter fare, smoothies, gourmet sandwiches\n• Rooftop Garden Café (Cancer Center): seasonal healthy options\n• Gift shop: packaged snacks, candies, beverages\n• Mobile food ordering: select local restaurants deliver to hospital lobby\n• Weekly food trucks (Tuesday/Thursday, 11am-2pm) in the visitor parking area during summer months\n\nHospital volunteers can provide information about nearby restaurants within walking distance."
            },
            {
                "regex": r"(?:do you|does the cafeteria) (?:offer|provide|have) (?:healthy|nutritious|wholesome) (?:food|meal|dining) (?:options|choices|alternatives)",
                "response": lambda match, user: "Yes, we emphasize healthy options in our cafeteria including:\n• 'Mindful Choice' items (under 500 calories, controlled sodium and fat) highlighted with green labels\n• Complete nutritional information displayed for all prepared foods\n• Fresh salad bar with numerous vegetables, lean proteins, and light dressing options\n• Steamed vegetables daily with no added salt or butter\n• Whole grain options for bread, pasta, and rice\n• Lean protein selections (baked, grilled, or roasted)\n• Fresh fruit available at all meals\n• Reduced-sugar dessert alternatives\n\nOur hospital dietitians regularly review and update menu items to ensure healthy, balanced options are always available."
            },
            {
                "regex": r"(?:what|which) (?:payment methods|forms of payment|ways to pay) (?:do you|does the cafeteria) (?:accept|take)",
                "response": lambda match, user: "Our cafeteria accepts multiple payment methods:\n• Cash\n• Credit cards (Visa, MasterCard, American Express, Discover)\n• Debit cards\n• Mobile payments (Apple Pay, Google Pay, Samsung Pay)\n• Employee badge payroll deduction (for staff only)\n• Meal vouchers (provided by some departments for patients' families in certain situations)\n• Contactless payment options\n\nWe also offer pre-loaded meal cards that can be purchased at the cashier station - these make convenient gifts for family members of patients with extended stays."
            },
            # Add 90+ more cafeteria patterns here
        ]

    def get_volunteer_patterns(self):
        return [
            {
                "regex": r"(?:how|what) (?:can|do) (?:I|someone) (?:become|apply to be|sign up as) (?:a|to be a) volunteer",
                "response": lambda match, user: "To become a hospital volunteer:\n1. Complete an online application at hospital.example.com/volunteer\n2. Attend an information session (held monthly)\n3. Complete an interview with our volunteer coordinator\n4. Pass a background check and health screening (including TB test and vaccinations)\n5. Attend orientation and training (4-6 hours)\n6. Commit to at least 4 hours per week for a minimum of six months\n\nThe application process typically takes 3-4 weeks. For questions, contact our Volunteer Services office at (555) 123-4567 ext. 5678 or volunteer@hospital.example.com."
            },
            {
                "regex": r"(?:what|which) (?:volunteer|volunteering) (?:opportunities|positions|roles|options) (?:are|do you) (?:available|offer|have)",
                "response": lambda match, user: "We offer diverse volunteer opportunities including:\n• Patient comfort ambassadors\n• Information desk/wayfinding\n• Gift shop assistants\n• Book/magazine cart service\n• Music program volunteers\n• Pet therapy program assistants\n• Emergency department support\n• Administrative/clerical assistance\n• Courier services\n• Spiritual care support\n• Surgery waiting room attendants\n• Maternity unit cuddlers (for babies with neonatal abstinence syndrome)\n• Hospice companions\n\nOpportunities vary based on hospital needs, volunteer availability, and special skills. We match volunteers to roles that fit their interests and abilities."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:requirements|qualifications|criteria) (?:for|to be|to become) (?:a|to be a) volunteer",
                "response": lambda match, user: "Basic volunteer requirements include:\n• Minimum age of 16 (certain areas require volunteers to be 18+)\n• Ability to commit to at least 4 hours weekly for 6+ months\n• Passing a background check\n• Completing health screening (TB test, flu vaccine during flu season)\n• COVID-19 vaccination\n• Proof of immunity to measles, mumps, rubella, and varicella\n• Ability to stand/walk for extended periods (some positions)\n• Strong communication skills\n• Professional demeanor and respect for privacy\n• Completion of orientation and training\n\nNo prior healthcare experience is required for most volunteer positions."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:volunteer|volunteering) (?:hours|schedule|shifts|times)",
                "response": lambda match, user: "Volunteer shifts are available 7 days a week:\n• Weekdays: 8:00 AM - 12:00 PM, 12:00 PM - 4:00 PM, 4:00 PM - 8:00 PM\n• Weekends: 9:00 AM - 1:00 PM, 1:00 PM - 5:00 PM\n• Special events: varied times\n\nMost volunteers serve one 4-hour shift weekly on a consistent schedule. Some departments have specific time requirements based on patient/visitor needs. Weekend and evening volunteers are especially needed. We work to accommodate volunteers' availability while meeting hospital needs. College students can arrange flexible schedules around classes."
            },
            {
                "regex": r"(?:are there|do you have|is there) (?:any|special) (?:benefits|perks|advantages) (?:for|to|of) (?:being a|becoming a) volunteer",
                "response": lambda match, user: "Volunteer benefits include:\n• Free parking during volunteer shifts\n• 20% discount in hospital cafeteria and gift shop\n• Annual recognition events and service awards\n• Letters of recommendation for school/employment\n• Professional references after 100 hours of service\n• Skills development and healthcare exposure\n• Access to continuing education opportunities\n• Invitation to hospital events\n• Volunteer newsletter subscription\n• Personal satisfaction of helping others\n• Social connections with other volunteers\n\nVolunteers also receive a uniform (vest or jacket) and name badge identifying them as hospital volunteers."
            },
            {
                "regex": r"(?:do|can) (?:volunteers|you) (?:work|help|assist) (?:directly|with|alongside) (?:patients|the sick|the ill)",
                "response": lambda match, user: "Yes, many volunteer roles involve direct patient interaction, including:\n• Visiting patients to provide companionship\n• Delivering flowers, mail, or reading materials\n• Assisting with meals and hydration\n• Providing hand massages or playing music\n• Supporting families in waiting areas\n• Helping with patient navigation and transport\n• Reading to or playing games with pediatric patients\n\nDirect patient care volunteers receive specialized training and are always supervised by clinical staff. Patient privacy and safety protocols must be strictly followed. Some patient care areas have additional requirements or restrictions for volunteers."
            },
            {
                "regex": r"(?:do you|is there|are there) (?:any|have|offer) (?:volunteer|volunteering) (?:opportunities|positions|roles) (?:for|available to) (?:teenagers|high school students|college students|youth)",
                "response": lambda match, user: "Yes, we have youth volunteer programs:\n\n• Teen Volunteer Program (ages 16-18):\n  - Information desk assistance\n  - Patient amenities delivery\n  - Clerical support\n  - Gift shop assistance\n  - Visitor escorts\n\n• Summer Healthcare Exploration Program (for high school juniors/seniors):\n  - Structured 8-week program\n  - Rotations through various departments\n  - Healthcare career seminars\n  - Mentorship opportunities\n\n• College Student Program:\n  - Pre-med/nursing student placements\n  - Department-specific opportunities\n  - Academic credit options (with school approval)\n\nAll youth volunteers must complete special orientation and parental consent forms."
            },
            {
                "regex": r"(?:is there|do you have|are there) (?:a|any) (?:time|hour|service) (?:commitment|requirement|obligation) (?:for|to be|to become) (?:a|to be a) volunteer",
                "response": lambda match, user: "Yes, we ask volunteers to commit to:\n• Minimum 4 hours per week\n• At least 6 months of service (100 hours minimum)\n• Consistent attendance at scheduled shifts\n• Advance notice for absences\n\nThis commitment allows volunteers to become familiar with their roles and develop meaningful relationships with staff and patients. It also ensures the hospital can rely on volunteer support. For those unable to make this commitment, we offer special event volunteering opportunities and our annual fundraising gala that require shorter time commitments."
            },
            {
                "regex": r"(?:how|what) (?:do|can) (?:volunteers|they) (?:help|assist|contribute to) (?:the|your) hospital",
                "response": lambda match, user: "Volunteers contribute significantly by:\n• Enhancing patient experience and satisfaction\n• Providing non-clinical support that frees staff for medical tasks\n• Offering comfort to patients and families during difficult times\n• Assisting with navigation and wayfinding (reducing missed appointments)\n• Supporting administrative functions\n• Raising funds through gift shop and fundraising events\n• Creating a more welcoming environment\n• Serving as hospital ambassadors in the community\n• Contributing thousands of service hours annually (over 25,000 hours last year)\n\nOur volunteers are integral to our mission of delivering compassionate, high-quality healthcare to our community."
            },
            {
                "regex": r"(?:can|do) (?:I|volunteers) (?:choose|select|pick) (?:which|what|the) (?:department|area|unit) (?:to|I want to|they) volunteer (?:in|at)",
                "response": lambda match, user: "During the application process, volunteers can indicate preferences for specific departments or types of service. We consider these preferences along with:\n• Hospital needs and available positions\n• Volunteer's skills and experience\n• Results of the volunteer interview\n• Schedule availability\n• Age restrictions for certain areas\n\nWhile we try to accommodate preferences, volunteers may initially be placed where needs are greatest. After completing 50 hours in an initial assignment, volunteers can request transfers to other areas if positions are available. Many volunteers discover unexpected satisfaction in roles they hadn't initially considered."
            },
            # Add 90+ more volunteer patterns here
        ]

    def get_feedback_patterns(self):
        return [
            {
                "regex": r"(?:how|where|what) (?:can|do) (?:I|patients|someone) (?:provide|give|submit|leave) (?:feedback|comments|suggestions|reviews)",
                "response": lambda match, user: "You can provide feedback through several channels:\n1. Patient satisfaction surveys (sent via email or mail after your visit)\n2. Comment cards available at all nursing stations and information desks\n3. Online feedback form at hospital.example.com/feedback\n4. Patient advocate office in the main lobby (Room A-105)\n5. Direct communication with unit managers or department directors\n6. Call our dedicated feedback line at (555) 123-9876\n7. Email to patientexperience@hospital.example.com\n\nAll feedback is reviewed regularly to identify improvement opportunities. We appreciate both positive comments and constructive suggestions."
            },
            {
                "regex": r"(?:how|what) (?:do|does) (?:the hospital|you) (?:use|do with|handle) (?:patient|feedback|survey) (?:feedback|comments|suggestions|reviews)",
                "response": lambda match, user: "We use patient feedback in several important ways:\n1. Regular review by our Patient Experience Committee\n2. Department-specific improvement initiatives based on trends\n3. Recognition programs for staff mentioned positively\n4. Training and education development\n5. Service recovery for specific concerns\n6. Quarterly reporting to hospital leadership and board\n7. Annual strategic planning for patient experience enhancement\n\nFeedback directly influences our policies, procedures, and facility improvements. We track feedback metrics over time to measure improvement and identify recurring issues that need systemic solutions."
            },
            {
                "regex": r"(?:what|which) (?:is|about) (?:the|your) (?:patient satisfaction|feedback|comment) (?:survey|questionnaire|form)",
                "response": lambda match, user: "Our patient satisfaction survey:\n• Is sent to patients within 1 week after discharge or outpatient visits\n• Takes approximately 10-15 minutes to complete\n• Covers key aspects of care including:\n  - Communication with doctors and nurses\n  - Responsiveness of staff\n  - Pain management\n  - Medication information\n  - Discharge instructions\n  - Facility cleanliness and quietness\n  - Overall rating and likelihood to recommend\n• Includes space for open-ended comments\n• Can be completed online, by mail, or by phone\n\nSurvey results are part of national quality metrics and help us compare our performance to other hospitals."
            },
            {
                "regex": r"(?:how|where|what) (?:can|do) (?:I|someone) (?:file|make|submit|report) (?:a|an) (?:complaint|grievance|concern)",
                "response": lambda match, user: "To file a complaint:\n1. Contact our Patient Advocate directly at (555) 123-9876\n2. Email concerns to patientadvocate@hospital.example.com\n3. Visit the Patient Advocate office (Main Lobby, Room A-105, open 8 AM-5 PM weekdays)\n4. Submit a written complaint to: Patient Relations, 123 Hospital Drive, City, State, ZIP\n5. Ask to speak with the department manager or supervisor while still in the facility\n\nAll complaints are taken seriously and investigated promptly. You'll receive acknowledgment within 2 business days and a full response within 7-10 days. Submitting a complaint will never adversely affect your care."
            },
            {
                "regex": r"(?:what|which) (?:is|about) (?:the|your) (?:complaint|grievance) (?:process|procedure|system)",
                "response": lambda match, user: "Our complaint resolution process includes:\n1. Initial documentation and acknowledgment (within 48 hours)\n2. Assignment to appropriate department for investigation\n3. Review of relevant medical records and interviews with involved staff\n4. Consultation with medical directors or administrators as needed\n5. Development of resolution plan\n6. Communication of findings and actions to the complainant\n7. Implementation of any necessary changes\n8. Follow-up to ensure satisfaction with resolution\n\nMost complaints are resolved within 7-10 business days. Complex issues may take longer. We'll keep you informed throughout the process and provide a clear explanation of our findings and actions taken."
            },
            {
                "regex": r"(?:who|what) (?:is|about) (?:the|your) (?:patient|hospital) (?:advocate|representative|ombudsman)",
                "response": lambda match, user: "Our Patient Advocates are trained professionals who:\n• Serve as liaisons between patients, families, and hospital staff\n• Help resolve concerns during your hospital stay before they become formal complaints\n• Explain hospital policies and procedures\n• Connect patients with appropriate resources\n• Facilitate communication with healthcare teams\n• Navigate complex healthcare situations\n• Ensure patients understand their rights\n• Follow up on specific requests or special needs\n\nPatient Advocates are available weekdays 8:00 AM to 5:00 PM in person (Room A-105) or by calling (555) 123-9876. For urgent concerns after hours, the nursing supervisor can assist."
            },
            {
                "regex": r"(?:can|will) (?:I|patients|someone) (?:get|receive) (?:a|an) (?:response|reply|answer|follow-up) (?:to|for|on) (?:my|their) (?:feedback|complaint|suggestion)",
                "response": lambda match, user: "Yes, you will receive a response to your feedback as follows:\n• For formal complaints: Written or verbal response within 7-10 business days\n• For general suggestions: Acknowledgment within 2-3 business days\n• For urgent concerns: Same-day response during business hours\n• For compliments: Staff recognition and thank you note\n\nWe take all feedback seriously and investigate thoroughly before responding. If your feedback requires extended review, we'll provide interim updates. Anonymous feedback is still reviewed and addressed, but we cannot provide direct responses unless contact information is provided."
            },
            {
                "regex": r"(?:how|what) (?:do|can) (?:I|patients|someone) (?:recognize|thank|appreciate|commend) (?:staff|employees|workers|doctors|nurses)",
                "response": lambda match, user: "To recognize exceptional staff members:\n1. Complete a 'Shining Star' card available at nursing stations and information desks\n2. Mention them by name in patient satisfaction surveys\n3. Send a letter/email to their department manager or hospital administration\n4. Post a positive review mentioning them on our hospital social media pages\n5. Nominate them for our quarterly Excellence Awards\n6. Make a donation in their honor through our Foundation\n\nStaff recognition is shared with the employee, their supervisor, and often highlighted in hospital newsletters. Your appreciation means a great deal to our dedicated healthcare professionals."
            },
            {
                "regex": r"(?:what|which) (?:information|details|data) (?:should|do|must) (?:I|patients) (?:include|provide) (?:in|when|with) (?:feedback|complaints|comments)",
                "response": lambda match, user: "When providing feedback, please include:\n• Your name and contact information (unless you wish to remain anonymous)\n• Patient name and room number/department visited (if applicable)\n• Date and time of your experience\n• Names of staff members involved (if known)\n• Specific details about the situation or concern\n• How the situation affected you or the patient\n• What would have improved your experience\n• Any attempts already made to resolve the issue\n• Your preferred method of contact for follow-up\n\nThe more specific information you provide, the better we can address your feedback or implement your suggestions."
            },
            {
                "regex": r"(?:is|will) (?:my|patient|the) (?:feedback|complaint|comment) (?:confidential|private|anonymous)",
                "response": lambda match, user: "We handle all feedback with appropriate confidentiality:\n• You may submit anonymous feedback if preferred\n• Identified feedback is kept in secure systems with limited access\n• Information is shared only with staff directly involved in addressing your concerns\n• Patient privacy is protected according to HIPAA regulations\n• Feedback data used for reporting is de-identified\n\nHowever, please note that providing contact information allows us to follow up with you and may lead to more thorough resolution. If you have specific confidentiality concerns, please mention them when submitting your feedback."
            },
            # Add 90+ more feedback patterns here
        ]

    def get_complaint_patterns(self):
        return [
            {
                "regex": r"(?:how|where|what) (?:can|do) (?:I|someone) (?:file|make|submit|report) (?:a|an) (?:complaint|grievance|concern)",
                "response": lambda match, user: "To file a complaint at our hospital:\n1. Speak directly with the unit/department manager where the issue occurred\n2. Contact our Patient Advocate Office at (555) 123-9870\n3. Email complaints to patientconcerns@hospital.example.com\n4. Complete a complaint form available at all nursing stations\n5. Visit the Patient Relations office in the main lobby (Room 105)\n6. Mail a written complaint to: Hospital Grievance Committee, 123 Medical Center Dr.\n\nAll complaints are logged, investigated, and addressed within our formal grievance process. You'll receive acknowledgment within 2 business days and a formal response within 7-10 days."
            },
            {
                "regex": r"(?:what|which) (?:is|about) (?:the|your) (?:complaint|grievance) (?:process|procedure|timeline)",
                "response": lambda match, user: "Our complaint process follows these steps:\n1. Receipt and documentation of your concern\n2. Assignment to appropriate department leader for investigation\n3. Review of medical records, interviews with staff, and analysis of policies\n4. Development of findings and resolution plan\n5. Response to complainant explaining findings and actions taken\n6. Implementation of any necessary improvements\n7. Follow-up to ensure resolution satisfaction\n\nSimple complaints are typically resolved within 7 days. Complex issues requiring extensive investigation may take up to 30 days, in which case you'll receive interim updates."
            },
            {
                "regex": r"(?:who|what|which) (?:handles|manages|processes|reviews) (?:complaints|grievances|concerns)",
                "response": lambda match, user: "Complaints are handled by a dedicated team including:\n• Patient Advocates who serve as your initial contact and guide through the process\n• Department Directors who investigate issues within their areas\n• The Grievance Committee for formal complaints, consisting of clinical leaders, quality improvement specialists, and administration\n• Risk Management for safety concerns or potential legal issues\n• The Chief Medical Officer for physician-related complaints\n• The Chief Nursing Officer for nursing care concerns\n\nThis multi-disciplinary approach ensures complaints receive appropriate expertise and thorough investigation."
            },
            {
                "regex": r"(?:what|which) (?:types of|kinds of|categories of) (?:complaints|grievances|concerns) (?:can|should|do) (?:I|patients) (?:report|file|submit)",
                "response": lambda match, user: "You can report any concerns about your hospital experience, including:\n• Quality of medical care or treatment decisions\n• Communication issues with healthcare providers\n• Medication errors or concerns\n• Pain management problems\n• Discharge planning inadequacies\n• Privacy violations or confidentiality concerns\n• Billing or financial issues\n• Facility cleanliness or environmental problems\n• Staff behavior or professionalism concerns\n• Wait times or access to care issues\n• Patient rights violations\n• Food quality or dietary accommodation issues\n\nNo concern is too small - we value all feedback that helps us improve patient care."
            },
            {
                "regex": r"(?:will|can|does) (?:filing|submitting|making) (?:a|my) (?:complaint|grievance|concern) (?:affect|impact|influence) (?:my|patient) (?:care|treatment)",
                "response": lambda match, user: "Absolutely not. We have strict policies prohibiting any form of retaliation against patients who file complaints. Your right to voice concerns is protected, and doing so will never negatively affect:\n• Quality of your medical care\n• Staff responsiveness or attitude\n• Access to services or treatments\n• Future appointments or admissions\n\nIn fact, we encourage feedback as essential to improving our care. Our complaint process operates independently from direct care providers to ensure objectivity and fairness. If you ever feel you're experiencing negative treatment after filing a complaint, please contact administration immediately."
            },
            {
                "regex": r"(?:can|is it possible to) (?:file|submit|make) (?:an|a) (?:anonymous|unnamed|unidentified) (?:complaint|grievance|concern)",
                "response": lambda match, user: "Yes, you can submit anonymous complaints through these channels:\n• Anonymous feedback box in main lobbies and clinic waiting areas\n• Online form at hospital.example.com/feedback (check 'remain anonymous' option)\n• Anonymous ethics hotline at (555) 123-0000\n\nHowever, please understand that anonymous complaints have limitations:\n• We cannot provide you with investigation results\n• We may be unable to gather additional information if needed\n• Some issues require specific details that might identify the situation\n\nWe protect the confidentiality of all complainants and encourage providing contact information whenever possible for more thorough resolution."
            },
            {
                "regex": r"(?:how|what) (?:do|does) (?:the hospital|you) (?:use|analyze|learn from) (?:complaints|grievances|concerns)",
                "response": lambda match, user: "Complaints provide valuable insights that we use to improve through:\n1. Tracking patterns and trends to identify systemic issues\n2. Monthly analysis of complaint data by our Quality Improvement Committee\n3. Development of action plans targeting recurring problems\n4. Staff education based on common misunderstandings or skill gaps\n5. Policy and procedure revisions when issues stem from flawed processes\n6. Technology improvements when systems contribute to problems\n7. Environmental modifications for facility-related concerns\n\nWe view each complaint as an opportunity to enhance patient experience and safety. A quarterly report summarizing complaint trends and improvements is shared with hospital leadership and our Board of Directors."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:my|patient) (?:rights|entitlements) (?:regarding|concerning|about) (?:complaints|grievances|concerns)",
                "response": lambda match, user: "Your rights regarding complaints include:\n• Right to file a complaint without fear of retaliation\n• Right to a prompt investigation and response\n• Right to a thorough explanation of findings\n• Right to appeal the resolution if unsatisfied\n• Right to file external complaints with regulatory agencies regardless of internal process\n• Right to assistance in filing a complaint if needed\n• Right to confidentiality throughout the process\n• Right to designate a representative to act on your behalf\n\nThese rights are outlined in our Patient Bill of Rights provided at admission and available at nursing stations or our website."
            },
            {
                "regex": r"(?:can|may|should) (?:I|someone) (?:file|submit|report) (?:a|an) (?:complaint|grievance|concern) (?:on behalf of|for) (?:someone else|another person|a family member)",
                "response": lambda match, user: "Yes, you can file a complaint on someone else's behalf in these circumstances:\n• You are the legal guardian or have medical power of attorney\n• You are the parent/guardian of a minor patient\n• The patient has provided written authorization for you to act on their behalf\n• The patient lacks capacity to file themselves\n\nWhen filing for someone else, please:\n• Provide documentation of your relationship/authority when applicable\n• Include both your contact information and the patient's details\n• Specify whether responses should go to you or the patient\n\nPatient privacy laws (HIPAA) may limit the information we can share with third parties without proper authorization."
            },
            {
                "regex": r"(?:what|which) (?:external|outside|regulatory) (?:agencies|organizations|authorities) (?:can|should) (?:I|someone) (?:contact|approach) (?:about|with|for) (?:complaints|grievances|concerns)",
                "response": lambda match, user: "If you're unsatisfied with our resolution, you may contact these external agencies:\n\n• State Department of Health: (800) 555-1234\n  For quality of care, patient safety, or hospital regulation concerns\n\n• The Joint Commission: (800) 994-6610 or complaint@jointcommission.org\n  For accreditation or safety standard concerns\n\n• Medicare Beneficiary Ombudsman: (800) MEDICARE\n  For Medicare patients with unresolved issues\n\n• State Medical Board: (800) 555-5678\n  For physician-specific complaints\n\n• Office for Civil Rights: (800) 368-1019\n  For HIPAA violations or discrimination concerns\n\nWe fully cooperate with all external investigations and respect your right to seek external review."
            },
            # Add 90+ more complaint patterns here
        ]

    def get_doctor_info_patterns(self):
        return [
            {
                "regex": r"(?:how|where|what) (?:can|do) (?:I|you) (?:find|locate|get) (?:information|details|data) (?:about|on|regarding) (?:doctors|physicians|specialists)",
                "response": lambda match, user: "You can find information about our physicians through multiple channels:\n1. Hospital website: hospital.example.com/find-a-doctor (searchable by name, specialty, languages spoken)\n2. Physician referral line: (555) 234-5678 (available 8am-8pm daily)\n3. Main information desk in hospital lobby\n4. Your insurance provider's directory (for network verification)\n5. Hospital directory app for smartphones\n6. Printed physician directories available at all clinic locations\n\nEach physician profile includes education, board certifications, specialties, languages spoken, accepting new patients status, and contact information."
            },
            {
                "regex": r"(?:how|what) (?:do|can) (?:I|patients) (?:choose|select|find) (?:the right|a good|an appropriate) doctor",
                "response": lambda match, user: "When selecting a doctor, consider these factors:\n• Board certification in their specialty\n• Experience treating your specific condition\n• Hospital affiliations and admitting privileges\n• Insurance network participation\n• Office location and hours\n• Communication style and language proficiency\n• Patient reviews and satisfaction scores\n• Whether they're accepting new patients\n• Gender preference (if important to you)\n• Recommendation from your primary care physician\n\nOur physician referral specialists at (555) 234-5678 can help match you with doctors based on your specific needs and preferences."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:doctor's|physician's|specialist's) (?:credentials|qualifications|certifications)",
                "response": lambda match, user: "Our physicians maintain rigorous credentials including:\n• Medical degree from accredited medical schools\n• Completion of residency training in their specialty\n• Board certification in their specialty areas\n• State medical licensure in good standing\n• Hospital privileges after credential verification\n• Ongoing continuing medical education\n• Regular peer review and quality assessment\n\nMany also have additional qualifications such as fellowship training, academic appointments, research publications, and leadership positions in medical societies. All credentials are verified through primary sources before granting hospital privileges."
            },
            {
                "regex": r"(?:how|what) (?:do|can) (?:I|patients) (?:make|schedule|book|get) (?:an appointment|a consultation) (?:with|to see) (?:a|the) doctor",
                "response": lambda match, user: "To schedule a doctor appointment:\n1. Call the physician's office directly (numbers listed in our directory)\n2. Use our online scheduling portal at hospital.example.com/appointments\n3. Call our central scheduling line at (555) 123-4567\n4. Use our hospital mobile app\n5. Visit the clinic in person\n\nWhen scheduling, please have your insurance information, reason for visit, and availability ready. Many routine appointments can be scheduled same-week, though specialist appointments may require longer wait times. For urgent needs, we offer same-day appointments at our urgent care centers."
            },
            {
                "regex": r"(?:which|what) (?:doctor|physician|specialist) (?:should|would) (?:I|someone) (?:see|visit|consult) (?:for|about) ([a-zA-Z\s]+)",
                "response": lambda match, user: self.recommend_specialist(match.group(1))
            },
            {
                "regex": r"(?:are|do) (?:your|the hospital's) (?:doctors|physicians|specialists) (?:accepting|taking) (?:new|additional) patients",
                "response": lambda match, user: "Most of our physicians are currently accepting new patients, though availability varies by specialty and individual doctor. Primary care physicians (family medicine, internal medicine, pediatrics) may occasionally close their practices to new patients when they reach capacity. Specialists typically accept new patients with appropriate referrals.\n\nOur website and physician referral line maintain up-to-date information on which doctors are accepting new patients. When calling our referral line at (555) 234-5678, we can specifically match you with physicians currently accepting new patients in your needed specialty."
            },
            {
                "regex": r"(?:what|which) (?:languages|foreign languages) (?:do|can|are) (?:your|the) (?:doctors|physicians|specialists) speak",
                "response": lambda match, user: "Our medical staff collectively speaks over 30 languages. Commonly available languages include:\n• Spanish (75+ physicians)\n• Mandarin and Cantonese (25+ physicians)\n• Hindi and Punjabi (20+ physicians)\n• Arabic (15+ physicians)\n• Russian (12+ physicians)\n• Vietnamese (10+ physicians)\n• Korean (8+ physicians)\n• French (20+ physicians)\n• Portuguese (5+ physicians)\n• American Sign Language (6+ physicians)\n\nOur physician directory allows searching by language. For languages not spoken by our physicians, we provide certified medical interpreters at no cost for all appointments when requested in advance."
            },
            {
                "regex": r"(?:how|what) (?:do|can) (?:I|patients) (?:check|verify|confirm) (?:doctor|physician|specialist) (?:reviews|ratings|feedback)",
                "response": lambda match, user: "You can check physician reviews and ratings through:\n1. Our hospital website's physician profiles (includes patient satisfaction scores)\n2. Press Ganey patient satisfaction survey results (available on request)\n3. Independent sites like Healthgrades, Vitals, or WebMD\n4. Your insurance provider's physician quality metrics\n\nWe publicly share our physicians' patient satisfaction scores related to:\n• Communication quality\n• Time spent with patients\n• Explanation of conditions and treatments\n• Likelihood to recommend\n\nOur Patient Experience department can also share aggregated feedback about specific physicians while maintaining individual patient confidentiality."
            },
            {
                "regex": r"(?:what|which) (?:insurance|insurance plans|insurance providers) (?:do|does) (?:your|the) (?:doctors|physicians|specialists) (?:accept|take)",
                "response": lambda match, user: "Most of our physicians accept major insurance plans including:\n• Blue Cross Blue Shield (all plans)\n• Aetna\n• Cigna\n• UnitedHealthcare\n• Medicare and Medicare Advantage\n• Medicaid and managed Medicaid plans\n• Tricare\n• Humana\n\nHowever, insurance acceptance varies by individual physician and practice. Always verify a specific doctor's insurance participation before scheduling by:\n1. Checking our online physician directory\n2. Calling the physician's office directly\n3. Contacting your insurance company\n\nThis verification is especially important for specialists and newer insurance products."
            },
            {
                "regex": r"(?:can|do) (?:I|patients) (?:get|request|have) (?:a|my|the) (?:second opinion|alternative opinion|different doctor)",
                "response": lambda match, user: "Yes, we fully support and encourage seeking second opinions for any diagnosis or treatment plan. To request a second opinion:\n1. Ask your current physician for a referral to another specialist\n2. Contact our physician referral line at (555) 234-5678 for appropriate specialists\n3. Check with your insurance about second opinion coverage\n\nMost insurance plans cover second opinions, especially for serious diagnoses or surgical recommendations. We can transfer relevant medical records and test results to the second opinion provider with your signed authorization. Our physicians respect patients' rights to seek additional perspectives on their healthcare decisions."
            },
            # Add 90+ more doctor info patterns here
        ]

    def get_facilities_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:facilities|amenities|services) (?:are|does|do you) (?:available|offer|provide) (?:at|in) (?:the|your) hospital",
                "response": lambda match, user: "Our hospital facilities include:\n\n• Emergency Department (24/7)\n• Inpatient units (medical, surgical, pediatric, maternity, ICU, NICU)\n• Outpatient surgery center\n• Comprehensive imaging center (X-ray, CT, MRI, ultrasound, PET)\n• Laboratory services\n• Heart and vascular center\n• Cancer center with infusion services\n• Rehabilitation center (PT, OT, speech therapy)\n• Women's health center\n• Behavioral health services\n• Wound care center\n• Sleep disorders lab\n• Cafeteria and coffee shop\n• Gift shop\n• Chapel/meditation room\n• Patient resource library\n• Free WiFi throughout the facility\n• ATM machines and vending areas"
            },
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors|patients) (?:find|locate|get to) (?:the|your) (?:cafeteria|dining area|food court)",
                "response": lambda match, user: "Our main cafeteria is located on the first floor of the South Wing (Building B). From the main entrance, follow the blue line on the floor or the overhead signs toward 'Dining Services.' The cafeteria is near elevator bank B, across from the gift shop. Hours are 6:30 AM - 8:00 PM daily, with hot meals served during breakfast (6:30-9:30 AM), lunch (11:00 AM-2:00 PM), and dinner (4:30-7:00 PM). Additional dining options include the coffee shop in the main lobby and vending areas throughout the hospital."
            },
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors|patients) (?:find|locate|get to) (?:the|your) (?:chapel|prayer room|meditation space|spiritual center)",
                "response": lambda match, user: "Our Interfaith Chapel is located on the second floor of the main building, near the central elevator bank. Follow the signs for 'Chapel' or the purple pathway on the floor. The chapel is open 24/7 to patients, families, and staff of all faiths. The space provides a quiet environment for prayer, meditation, or reflection. Adjacent to the main chapel are smaller rooms designated for specific faith traditions. Our Spiritual Care department staff and volunteers are available from 7:00 AM to 9:00 PM daily, with on-call chaplains available overnight for urgent spiritual needs."
            },
            {
                "regex": r"(?:do you have|is there|are there) (?:ATMs|cash machines|banking services) (?:in|at|inside) (?:the|your) hospital",
                "response": lambda match, user: "Yes, we have ATMs located at:\n1. Main lobby near the information desk\n2. First floor near the cafeteria entrance\n3. Outside the gift shop\n4. Emergency Department waiting area\n\nAll ATMs accept major banking networks (Plus, Cirrus, Star, etc.) and dispense in $20 denominations. Transaction fees may apply depending on your bank. The ATMs in the main lobby and cafeteria areas also accept deposits for FirstBank and National Credit Union customers."
            },
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors|patients) (?:find|locate|get to) (?:the|your) (?:gift shop|store|gift store)",
                "response": lambda match, user: "Our gift shop is located in the main lobby, to the right of the information desk. It offers flowers, greeting cards, toiletries, snacks, magazines, books, toys, and hospital-branded items. Hours are Monday-Friday 9:00 AM to 7:00 PM, and weekends 11:00 AM to 5:00 PM. The shop is operated by our hospital auxiliary volunteers, and all proceeds support patient care programs. If you can't visit in person, you can order items by calling (555) 123-4569, and they can be delivered to patient rooms (except ICU and isolation rooms)."
            },
            {
                "regex": r"(?:do you have|is there|are there) (?:wifi|internet|wireless) (?:access|connection|service) (?:in|at|throughout) (?:the|your) hospital",
                "response": lambda match, user: "Yes, we provide complimentary WiFi throughout the entire hospital. To connect:\n1. Select \"Hospital_Guest\" from your device's WiFi networks\n2. Open your web browser, which will redirect to our terms of service page\n3. Accept the terms of use\n4. Begin browsing\n\nThe guest network has reasonable speed for basic browsing, email, and video calls. However, large downloads or streaming may be limited for bandwidth management. For security reasons, the guest WiFi is separate from our clinical systems. If you experience connection difficulties, please ask at the information desk or call our IT help desk at extension 4357 (HELP) from any hospital phone."
            },
            {
                "regex": r"(?:do you have|is there|are there) (?:a|any) (?:pharmacy|hospital pharmacy|drug store) (?:in|at|inside) (?:the|your) hospital",
                "response": lambda match, user: "Yes, our outpatient pharmacy is located on the first floor of the main building, adjacent to the main exit. Hours are:\n• Monday-Friday: 8:00 AM to 9:00 PM\n• Saturday: 9:00 AM to 6:00 PM\n• Sunday: 10:00 AM to 5:00 PM\n\nThe pharmacy fills prescriptions from our hospital physicians as well as outside providers. They offer bedside delivery of discharge medications, medication counseling, a full range of over-the-counter products, and accept most insurance plans. For refills or questions, call (555) 234-6789. We also offer a home delivery service for maintenance medications."
            },
            {
                "regex": r"(?:do you have|is there|are there) (?:interpreter|translation|language) (?:services|assistance|help) (?:in|at|available at) (?:the|your) hospital",
                "response": lambda match, user: "Yes, we provide comprehensive interpreter services 24/7 at no cost to patients:\n\n• In-person interpreters for Spanish, Mandarin, Vietnamese, Russian, and ASL (available with advance notice)\n• Video remote interpreting for 35+ languages (available within minutes)\n• Phone interpretation for 200+ languages (immediate access)\n\nTo request an interpreter, inform any staff member or call (555) 123-4500. For scheduled appointments, please request interpretation services when booking to ensure availability. All medical interpreters are certified in healthcare interpretation and bound by confidentiality requirements."
            },
            {
                "regex": r"(?:what|which) (?:accessibility|handicap|disability|ADA) (?:features|accommodations|services) (?:are|does|do you) (?:available|offer|provide)",
                "response": lambda match, user: "Our accessibility accommodations include:\n• Wheelchair-accessible entrances, elevators, and restrooms throughout\n• Designated accessible parking spaces near all entrances\n• Wheelchairs available at all entrances (free of charge)\n• Patient rooms with ADA-compliant bathrooms and fixtures\n• Adjustable-height beds in all rooms\n• ASL interpreters and video remote interpreting\n• Telecommunication devices for deaf patients (TDD/TTY)\n• Braille signage and raised lettering on room signs\n• Service animal accommodations\n• Bariatric equipment and furniture in designated areas\n\nFor specific accommodation needs, please contact our Patient Advocacy office at (555) 123-9876 prior to your visit."
            },
            {
                "regex": r"(?:do you have|is there|are there) (?:a|any) (?:patient|health|medical) (?:resource|education|learning) (?:center|library) (?:in|at) (?:the|your) hospital",
                "response": lambda match, user: "Yes, our Patient Resource Center is located on the first floor near the central elevators (Room 1420). It provides:\n• Medical reference books and simplified health guides\n• Computer stations with access to reliable health information websites\n• Printed materials on common conditions and treatments\n• Educational videos and interactive learning modules\n• Information about community health resources\n• Assistance researching medical conditions\n\nA health librarian staffs the center Monday-Friday, 9:00 AM to 4:00 PM, to help find relevant information. The space is open to patients, families, and community members. Materials cannot be checked out but can be copied or digitally sent to your email."
            },
            # Add 90+ more facilities patterns here
        ]

    def get_covid_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:covid|covid-19|coronavirus) (?:policies|protocols|procedures)",
                "response": lambda match, user: "Our current COVID-19 policies include:\n• Universal masking required in high-risk areas (oncology, transplant, NICU)\n• Masks recommended but optional in other clinical areas\n• Symptom screening at main entrances\n• Hand hygiene stations throughout facilities\n• Enhanced cleaning protocols in all areas\n• Respiratory etiquette (covering coughs/sneezes) encouraged\n• Testing for symptomatic patients and prior to certain procedures\n• Telehealth options for appropriate appointments\n• Isolation protocols for COVID-positive patients\n\nThese policies are regularly updated based on community transmission levels, CDC guidance, and public health recommendations."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:visitor|visitation) (?:policy|restrictions|rules) (?:for|during|related to) (?:covid|covid-19|coronavirus)",
                "response": lambda match, user: "Our current COVID-19 visitor policies:\n• Two visitors allowed per patient in most areas\n• Visitors must be 12+ years old (except for special circumstances)\n• No visitors with COVID-19 symptoms, positive test within 10 days, or known exposure within 14 days\n• Specialized units (ICU, Maternity, Pediatrics) may have modified policies\n• COVID-positive patients limited to one designated visitor per day\n• Video visits encouraged as an alternative when in-person visits aren't possible\n\nThese policies are subject to change based on community transmission levels. Current policies are posted at entrances and on our website."
            },
            {
                "regex": r"(?:do you|does the hospital) (?:offer|provide|have) (?:covid|covid-19|coronavirus) (?:testing|test|screening)",
                "response": lambda match, user: "Yes, we offer COVID-19 testing through multiple channels:\n\n• Emergency Department: For symptomatic patients requiring emergency care\n• Urgent Care Centers: For symptomatic patients needing evaluation\n• Pre-procedure Testing: Required 1-3 days before certain procedures\n• Outpatient Testing Center: By appointment with doctor's order\n• Drive-through Testing: By appointment (check website for current locations/times)\n\nWe currently use PCR tests (results in 24-48 hours) and rapid antigen tests (results in 15-30 minutes) depending on the clinical situation. Most insurance plans cover medically-necessary testing. Please check with your insurance regarding coverage for travel or general screening tests."
            },
            {
                "regex": r"(?:do you|does the hospital) (?:offer|provide|have) (?:covid|covid-19|coronavirus) (?:vaccine|vaccination|immunization)",
                "response": lambda match, user: "Yes, we offer COVID-19 vaccines including primary series and updated boosters. Vaccines are available at:\n• Our hospital pharmacy (walk-ins welcome Monday-Friday 9am-4pm)\n• All primary care clinic locations (by appointment)\n• Dedicated vaccine clinics (schedule varies - check our website)\n\nWe currently offer the latest formulations authorized by the FDA/CDC. COVID-19 vaccines can be administered simultaneously with other vaccines, including flu shots. Vaccines are provided at no cost regardless of insurance status. To schedule, call (555) 678-9012 or visit our website at hospital.example.com/covid-vaccines."
            },
            {
                "regex": r"(?:what|which) (?:covid|covid-19|coronavirus) (?:precautions|safety measures|protections) (?:are|is|do) (?:in place|being taken|implemented)",
                "response": lambda match, user: "Our comprehensive COVID safety measures include:\n\n• HEPA filtration and enhanced ventilation throughout facilities\n• Negative pressure rooms for aerosol-generating procedures\n• Rigorous environmental cleaning with hospital-grade disinfectants\n• UV light disinfection for high-risk areas\n• Physical barriers at registration desks and other interfaces\n• Reduced density in waiting areas\n• Separate pathways for COVID-positive patients\n• PPE supplies maintained at optimal levels\n• Regular staff testing protocols\n• Vaccination requirements for healthcare personnel\n• Ongoing infection prevention training\n\nThese measures are continually evaluated and updated based on emerging evidence and expert guidance."
            },
            {
                "regex": r"(?:what|how) (?:should|do) (?:I|patients) (?:prepare|get ready) (?:for|before) (?:a|my|an) (?:appointment|procedure|surgery|visit) (?:during|in|with) (?:covid|covid-19|coronavirus)",
                "response": lambda match, user: "To prepare for your appointment during COVID-19:\n\n1. Monitor for symptoms before your visit (fever, cough, shortness of breath)\n2. Reschedule if you develop symptoms or test positive within 10 days\n3. Consider wearing a mask, especially if immunocompromised\n4. Arrive at your scheduled time (not early) to reduce waiting room density\n5. Complete pre-registration and forms online when possible\n6. Limit accompanying visitors to essential support persons\n7. Be prepared for symptom screening upon arrival\n\nSome procedures may require COVID testing 24-72 hours before; your provider will advise if this applies to your situation."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:symptoms|signs) (?:of|for) (?:covid|covid-19|coronavirus)",
                "response": lambda match, user: "Common COVID-19 symptoms include:\n• Fever or chills\n• Cough\n• Shortness of breath or difficulty breathing\n• Fatigue\n• Muscle or body aches\n• Headache\n• New loss of taste or smell\n• Sore throat\n• Congestion or runny nose\n• Nausea or vomiting\n• Diarrhea\n\nSymptoms typically appear 2-14 days after exposure and range from mild to severe. Older adults and those with underlying medical conditions may develop more serious complications. If you experience emergency warning signs (trouble breathing, persistent chest pain, confusion, inability to stay awake, bluish lips/face), seek emergency medical care immediately."
            },
            {
                "regex": r"(?:what|which) (?:should|do) (?:I|someone) (?:do|if) (?:I have|with|experiencing) (?:covid|covid-19|coronavirus) (?:symptoms|exposure)",
                "response": lambda match, user: "If you have COVID-19 symptoms or exposure:\n\n1. Isolate from others immediately\n2. Get tested (PCR or rapid antigen test)\n   • Our Urgent Care centers offer walk-in testing\n   • Testing center appointments: (555) 678-9012\n3. Wear a high-quality mask if you must be around others\n4. Contact your healthcare provider, especially if you're high-risk\n5. Monitor your symptoms closely\n\nSeek emergency care immediately for severe symptoms like difficulty breathing, persistent chest pain, confusion, inability to wake/stay awake, or bluish lips/face. Call ahead to emergency facilities if possible to notify them of your COVID status."
            },
            {
                "regex": r"(?:how|what) (?:are|is) (?:covid|covid-19|coronavirus) (?:patients|cases) (?:treated|managed|handled) (?:at|in) (?:the|your) hospital",
                "response": lambda match, user: "Our COVID-19 patient care includes:\n\n• Dedicated COVID units with specialized ventilation systems\n• Treatment protocols based on latest clinical evidence\n• FDA-authorized antivirals (Paxlovid, remdesivir) for eligible patients\n• Monoclonal antibody treatments for qualifying patients\n• Respiratory support ranging from oxygen supplementation to ventilation\n• Prone positioning protocols to improve oxygenation\n• Multidisciplinary care teams including infectious disease specialists\n• Critical care expertise for severe cases\n• Continuous monitoring of vital signs and oxygen levels\n• Remote monitoring programs for recovering outpatients\n\nTreatment approaches are personalized to each patient's condition, risk factors, and disease progression."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:long covid|post-covid|long-haul covid) (?:services|treatment|care|clinic)",
                "response": lambda match, user: "Our Post-COVID Recovery Program provides comprehensive care for long COVID symptoms including:\n\n• Multidisciplinary evaluation by pulmonology, cardiology, neurology, and rehabilitation medicine\n• Personalized treatment plans addressing specific symptoms\n• Pulmonary function testing and rehabilitation\n• Cardiac evaluation and monitoring\n• Cognitive assessment and rehabilitation\n• Physical and occupational therapy\n• Mental health support and counseling\n• Sleep disorder evaluation and management\n• Nutrition guidance\n• Support groups and educational resources\n\nReferrals can come from primary care or specialists. For more information or appointments, call our Post-COVID Clinic at (555) 234-5678."
            },
            # Add 90+ more COVID-19 patterns here
        ]

    def get_contact_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:is|are) (?:the|your) (?:contact|phone|telephone) (?:number|info|information|details)",
                "response": lambda match, user: (
                    "Our main hospital contact numbers:\n\n"
                    "• Main Switchboard: (555) 123-4500\n"
                    "• Patient Information: (555) 123-4501\n"
                    "• Appointment Scheduling: (555) 123-4567\n"
                    "• Billing Department: (555) 987-6543\n"
                    "• Medical Records: (555) 123-4545\n"
                    "• Patient Advocacy: (555) 123-9876\n"
                    "• Emergency Department: (555) 911-7000\n"
                    "• Physician Referral Line: (555) 234-5678\n\n"
                    "For direct connections to specific departments or patient rooms, please call our main switchboard. Our operators are available 24/7 to direct your call appropriately."
                )
            },
        ]

    def get_working_hours_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:working|operating|business) hours(?: for| of)? (?:the hospital|you)?",
                "response": lambda match, user: "Our hospital operates 24/7 for inpatient care and emergency services. Specific department hours include:\n• Main Hospital: Open 24/7\n• Emergency Department: Open 24/7\n• Outpatient Clinics: Monday-Friday, 8:00 AM - 5:00 PM\n• Laboratory: Monday-Friday, 6:00 AM - 6:00 PM; Saturday, 7:00 AM - 12:00 PM\n• Imaging Services: Monday-Friday, 7:00 AM - 7:00 PM; Saturday, 8:00 AM - 2:00 PM\n• Pharmacy: Monday-Friday, 8:00 AM - 9:00 PM; Saturday, 9:00 AM - 6:00 PM; Sunday, 10:00 AM - 5:00 PM\n\nFor specific department hours or holiday schedules, please call (555) 123-4500."
            },
            {
                "regex": r"(?:what|which) (?:are|is) (?:the|your) (?:holiday|weekend) hours(?: for| of)? (?:the hospital|you)?",
                "response": lambda match, user: "Our hospital remains open 24/7 during holidays for inpatient and emergency care. However, outpatient services and administrative offices may have modified hours:\n• Emergency Department: Open 24/7\n• Inpatient Units: Open 24/7\n• Outpatient Clinics: Closed on major holidays\n• Pharmacy: Open 10:00 AM - 2:00 PM on holidays\n• Laboratory: Closed on major holidays\n• Imaging Services: Emergency imaging only\n\nPlease contact specific departments or visit hospital.example.com/holidays for detailed holiday hours."
            },
            # Add 90+ more working hours patterns here
        ]

    def get_parking_patterns(self):
        return [
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors|patients) (?:park|find parking|locate parking) at the hospital",
                "response": lambda match, user: "Visitor parking is available in the following areas:\n• Main Visitor Garage (adjacent to the main entrance): $2/hour, $10 daily max\n• Surface Lot B (near Emergency Department): $1/hour, $8 daily max\n• Valet Parking (Main Entrance): $5 for up to 4 hours, $15 daily max; available 6:00 AM - 8:00 PM weekdays\n• Free parking for patients with extended stays (passes available at the information desk)\n\nAll parking areas are open 24/7. Accessible parking spaces are available near all major entrances. Contact Security at (555) 123-7890 for assistance."
            },
            # Add 90+ parking patterns here
        ]

    def get_directions_patterns(self):
        return [
            {
                "regex": r"(?:where|how) (?:can|do) (?:I|visitors|patients) (?:find|get|locate) (?:directions|maps|routes) to the hospital",
                "response": lambda match, user: "For directions to our hospital:\n• Address: 123 Medical Center Drive, Healthville, ST 12345\n• GPS: Enter 'Main Hospital Entrance' or 'Emergency Department Entrance' for precise navigation\n• Public Transit: Bus routes 10, 15, and 22 stop at the main entrance\n• Driving Directions: Visit hospital.example.com/directions for turn-by-turn instructions\n• Parking: Visitor Garage (Main Entrance) or Surface Lot B (Emergency Department)\n\nIf you need additional assistance, call our main switchboard at (555) 123-4500."
            },
            # Add 90+ directions patterns here
        ]

    def get_greeting_patterns(self):
        return [
            {
                "regex": r"(?:hello|hi|hey|good morning|good afternoon|good evening)",
                "response": lambda match, user: "Hello! How can I assist you with our hospital services today?"
            },
            # Add more greeting patterns here
        ]

    def get_goodbye_patterns(self):
        return [
            {
                "regex": r"(?:goodbye|bye|see you|take care|farewell)",
                "response": lambda match, user: "Goodbye! Please don't hesitate to contact us if you need further assistance."
            },
            # Add more goodbye patterns here
        ]

    def get_thanks_patterns(self):
        return [
            {
                "regex": r"(?:thank you|thanks|much appreciated|grateful)",
                "response": lambda match, user: "You're welcome! I'm glad I could help. Let us know if you have any other questions."
            },
            # Add more thanks patterns here
        ]

    def get_general_patterns(self):
        return [
            {
                "regex": r"(?:what|which) (?:services|departments|units) (?:are|is) (?:available|offered) at the hospital",
                "response": lambda match, user: "We offer a wide range of services, including emergency care, surgery, maternity, pediatrics, cardiology, oncology, rehabilitation, and more. For a full list of departments, please visit hospital.example.com/services or call (555) 123-4500."
            },
            # Add more general patterns here
        ]

    def get_spelling_corrections(self):
        return {
            "apointment": "appointment",
            "schedual": "schedule",
            "medecine": "medicine",
            "pharmacyy": "pharmacy",
            "insurence": "insurance",
            "hospitol": "hospital",
            "emerjency": "emergency",
            "visiter": "visitor",
            "cafateria": "cafeteria",
            "volenteer": "volunteer",
            "feedbak": "feedback",
            "complant": "complaint",
            "dischagre": "discharge",
            "admisson": "admission",
        }

In [19]:
if __name__ == "__main__":
    chatbot = HospitalChatbot()

    print("Welcome to the Hospital Chatbot! How can I assist you today?")
    print("Type 'exit' to end the chat.\n")

    user_name = input("Before we begin, may I have your name? ").strip()
    if not user_name:
        user_name = None  # Handle case where user doesn't provide a name

    while True:
        user_input = input("\nYou: ")

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Thank you for using the hospital chatbot. Take care!")
            break

        response = chatbot.process_query(user_input, user_name)
        print(f"Chatbot: {response}")


Welcome to the Hospital Chatbot! How can I assist you today?
Type 'exit' to end the chat.

Before we begin, may I have your name? Waleed Usman

You: what are the process to book an appointment with doctor
Chatbot: To schedule an appointment at our hospital, you can:
1. Call our appointment line at (555) 123-4567
2. Use our patient portal at hospital.example.com
3. Visit the front desk in person

You'll need your insurance information and patient ID if you're a returning patient.

You: can you book an appointment with Dr Noor
Chatbot: I'm not sure I understand your question about the hospital. Could you please rephrase it?

You: can you book an appointment with Doctor Ali
Chatbot: Yes, you can schedule an appointment with Doctor Ali. Please call our appointment desk at (555) 123-4567 or use our online patient portal. When would you prefer to see Doctor Ali?

You: what documents should i bring for an appointment
Chatbot: Please bring the following to your appointment:
1. Photo ID
2. Insu